# Testing methods from `utils/autocorr.py`

## Imports

In [1]:
import datetime
import os
import sys
import time

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from rich.console import Console
from rich.theme import Theme

project_dir = os.path.abspath('/lus/grand/projects/DLHMC/l2hmc-qcd/l2hmc-qcd/')
if project_dir not in sys.path:
    sys.path.append(project_dir)

sns.set_palette('bright')
%load_ext autoreload
%autoreload 2

console = Console(theme=Theme({"repr.number": "#ff79ff", "repr.path": '#f92672'}),
                  width=180, log_time=True, log_time_format='[%X]', log_path=False)
#%matplotlib notebook
plt.style.use('default')
sns.set_context('talk')
sns.set_style('whitegrid')
sns.set_palette('bright')
%matplotlib widget

GREEN = '#87ff00'
PINK = '#F92672'
BLUE = '#007dff'
YELLOW = '#FFFF00'
RED = '#FF4050'


Bad key "text.kerning_factor" on line 4 in
/lus/grand/projects/DLHMC/tf_master-2021-03-02/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
%matplotlib widget

In [3]:
plt.style.use('default')
sns.set_context('talk')
sns.set_style('whitegrid')
sns.set_palette('bright')

In [4]:
def bootstrap(x, *, Nboot, binsize):
    boots = []
    x = x.reshape(-1, binsize, *x.shape[1:])
    for i in range(Nboot):
        boots.append(np.mean(x[np.random.randint(len(x), size=len(x))], axis=(0, 1)))
        
    return np.mean(boots), np.std(boots)

In [5]:
from copy import deepcopy

def print_l2hmc_data(l2hmc_data, make_copy=False):
    l2hmc_data_ = {}
    for idx, (beta, val) in enumerate(l2hmc_data.items()):
        if make_copy:
            l2hmc_data_[beta] = {}
            
        console.rule(f'[{idx}] :: {beta}')
        for jdx, (tlen, data) in enumerate(val.items()):
            if tlen < 1.:
                continue
            run_dir = data['params']['run_dir']
            tint_avg = np.mean(data['tint'][-1, :])
            tint_err = np.std(data['tint'][-1, :])
            bi = data['beta_init']
            bf = data['beta_final']
            if make_copy:
                l2hmc_data_[beta][tlen] = deepcopy(data)
                l2hmc_data_[beta][tlen].update({
                    'beta_init': int(bi),
                    'beta_final': int(bf),
                    'tint_avg': tint_avg,
                    'tint_err': tint_err,
                })

            console.log(f'[{idx}::{jdx}] :: beta: {beta}, bi: {bi}, bf: {bf}, traj_len: {tlen}, tint[-1]: {tint_avg} +/- {tint_err}', style=GREEN)
            console.log(f'[{idx}::{jdx}] :: run_dir: {run_dir}', style=GREEN)
            console.log(80*'-', style=RED)
            
    return l2hmc_data_ if make_copy else l2hmc_data

In [6]:
def search_for_annealing_schedule(run_dir):
    inference_configs_file = os.path.join(run_dir, 'inference_configs.z')
    configs = io.loadz(os.path.join(inference_configs_file))
    log_dir = configs.get('log_dir', None)
    beta0 = None
    beta1 = None
    if log_dir is not None:
        sched_file = os.path.join(log_dir, 'annealing_schedule.txt')
        if os.path.isfile(sched_file):
            schedule = np.loadtxt(sched_file, delimiter=',')
            beta0 = schedule[0]
            beta1 = schedule[1]
    return beta0, beta1

def get_annealing_schedule(run_dir):
    sched_file = os.path.join(run_dir, 'annealing_schedule.txt')
    if os.path.isfile(sched_file):
        sched = np.loadtxt(sched_file, delimiter=',')
        bi = sched[0]
        bf = sched[1]
        
    else:
        try:
            bi, bf = search_for_annealing_schedule(run_dir)
        except FileNotFoundError:
            bi = None
            bf = None
        
    if bi is None and bf is None:
        tail = str(run_dir).find('GaugeModel_logs')
        run_dir = run_dir[tail:]

        bidx = str(run_dir).find('bi')
        bfdx = str(run_dir).find('bf')

        bistr = run_dir[bidx:bidx+3]
        bfstr = run_dir[bfdx:bfdx+3]

        bi = float(bistr.lstrip('bi'))
        bf = float(bfstr.lstrip('bf'))
    
    astr = r"$\beta:$" + f' {bi}' + r"$\rightarrow$" + f'{bf}'
    if bi is not None:
        bi = int(bi)
        
    if bf is not None:
        bf = int(bf)
    
    return astr, (bi, bf)

In [7]:
from utils.plotting_utils import plot_autocorrs_vs_draws

def recompute_autocorrs_from_run_dir(run_dir):
    data_dir = os.path.join(run_dir, 'run_data')
    plots_dir = os.path.join(run_dir, 'plots')
    
    charges = io.loadz(os.path.join(data_dir, 'charges.z'))
    run_params = io.loadz(os.path.join(run_dir, 'run_params.z'))
    
    beta = run_params.get('beta', None)
    xeps = run_params.get('xeps', None)
    veps = run_params.get('veps', None)
    lf = run_params.get('num_steps', None)
    
    traj_len = np.sum(xeps)
    xeps_avg = np.mean(xeps)
    veps_avg = np.mean(veps)

    run_params.update({
        'xeps': xeps,
        'veps': veps,
        'traj_len': traj_len,
        'xeps_avg': xeps_avg,
        'veps_avg': veps_avg,
        'eps_avg': (xeps_avg + veps_avg) / 2.,
    })

    tint_dict, _ = plot_autocorrs_vs_draws(charges, num_pts=20, nstart=1000,
                                           therm_frac=0.2, out_dir=plots_dir, lf=lf)
    
    tint_data = {
        'beta': 6.,
        'run_dir': run_dir,
        'traj_len': traj_len,
        'run_params': run_params,
        'eps': run_params['eps_avg'],
        'lf': lf,
        'narr': tint_dict['narr'],
        'tint': tint_dict['tint'],
    }

    tint_file = os.path.join(run_dir, 'tint_data.z')
    io.savez(tint_data, tint_file, 'tint_data')
    
    return tint_data

tf.__version__: 2.5.0


## `HMC` dirs:

In [8]:
import glob
from pathlib import Path

import utils.file_io as io
import utils.autocorr as autocorr

from config import LOGS_DIR


base_dirs_hmc = [
    os.path.abspath('/lus/grand/projects/DLHMC/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs/'),
    os.path.abspath('/lus/theta-fs0/projects/DLHMC/thetaGPU/inference/hmc_2020_12_20/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs/'),
    os.path.abspath('/lus/theta-fs0/projects/DLHMC/the9taGPU/inference/hmc_2021_01_04/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs/'),
    os.path.abspath('/lus/theta-fs0/projects/DLHMC/thetaGPU/inference/hmc_2021_01_05/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs/'),
    os.path.abspath('/lus/theta-fs0/projects/DLHMC/thetaGPU/inference/hmc/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs/'),
    os.path.abspath('/lus/theta-fs0/projects/DLHMC/thetaGPU/inference/hmc_2020-12-15/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs'),
    os.path.abspath('/lus/theta-fs0/projects/DLHMC/thetaGPU/inference/hmc_2020_12_20/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs/'),
    os.path.abspath('~/thetaGPU/inference/hmc/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs/'),
    os.path.abspath('~/thetaGPU/inference/hmc_2020-12-15/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs'),
    os.path.abspath('~/thetaGPU/inference/hmc_2020_12_20/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs/'),
    os.path.abspath('~/inference/hmc_2021_01_04/l2hmc-qcd/logs/GaugeModel_logs/hmc_logs/'),
    os.path.abspath(Path('~/thetaGPU/inference/')),
    os.path.abspath('/lus/grand/projects/DLHMC/l2hmc-qcd/'),
    os.path.abspath(Path('~/thetaGPU/inference/')),
    #os.path.abspath(Path('/lus/l2hmc-qcd/logs/')),
]

hmc_dirs = []
for d in base_dirs_hmc:
    if os.path.isdir(d) and 'bad' not in str(d):
        console.log(f'Looking in {d}...')
        hmc_dirs += [x for x in Path(d).rglob('*HMC_L16*') if x.is_dir()]
        console.log(f'len(hmc_dirs): {len(hmc_dirs)}')
        
hmc_dirs = np.unique(hmc_dirs)

In [9]:
len(hmc_dirs)

301

In [10]:
base_hmc = [
    os.path.abspath('/lus/grand/projects/DLHMC/'),
    os.path.abspath('/home/foremans/thetagpu/inference'),
    os.path.abspath('/home/foremans/thetagpu/training/'),
    os.path.abspath('/home/foremans/thetagpu/l2hmc-qcd/'),
]

hmc_dirs = list(hmc_dirs)
for d in base_hmc:
    console.log(f'{d}')
    if os.path.isdir(d) and 'bad' not in str(d):
        console.log(f'Looking in: {d}...')
        hmc_dirs += [x for x in Path(d).rglob('*HMC_L16*') if x.is_dir()]
        console.log(f'len(hmc_dirs): {len(hmc_dirs)}')
        
hmc_dirs = np.unique(hmc_dirs)

In [11]:
len(hmc_dirs)  # 547 (2021-02-25 15:03:32)

459

## `L2HMC` dirs:

In [12]:
from pathlib import Path
import utils.file_io as io
import utils.autocorr as acr

def _look(p, s, conds=None):
    matches = [x for x in Path(p).rglob(f'*{s}*')]
    if conds is not None:
        if isinstance(conds, (list, tuple)):
            for cond in conds:
                matches = [x for x in matches if cond(x)]
        else:
            matches = [x for x in matches if cond(x)]
    return matches

base_dirs_l2hmc = [
    os.path.abspath('/lus/theta-fs0/projects/DLHMC/thetaGPU/training'),
    os.path.abspath('/lus/grand/projects/DLHMC/training/'),
    os.path.abspath('/lus/grand/projects/DLHMC/training/net_weights_tests/'),
    os.path.abspath('/lus/grand/projects/DLHMC/training/annealing_schedules/'),
    os.path.abspath('/lus/grand/projects/DLHMC/l2hmc-qcd/logs/GaugeModel_logs/'),
    #os.path.abspath('/lus/grand/projects/DLHMC/l2hmc-qcd/logs/GaugeModel_logs/2021_02'),
    os.path.abspath('/lus/grand/projects/DLHMC/l2hmc-qcd/logs/GaugeModel_logs/'),
    os.path.abspath('/lus/grand/projects/DLHMC/l2hmc-qcd/logs/GaugeModel_logs/l2hmc_logs/'),
    #os.path.abspath('/Users/saforem2/thetaGPU/training/'),
    #os.path.abspath('/Users/saforem2/grand/projects/DLHMC/training/'),
    #os.path.abspath('/Users/saforem2/grand/projects/DLHMC/l2hmc-qcd/logs/GaugeModel_logs/l2hmc_logs/'),
    #os.path.abspath('/Users/saforem2/grand/projects/DLHMC/training/annealing_schedules/'),
    #os.path.abspath('/Users/saforem2/grand/projects/DLHMC/l2hmc-qcd/logs/GaugeModel_logs/2021_02')
]
l2hmc_dirs = []
for d in base_dirs_l2hmc:
    console.log(f'Looking in: {d}...')
    if os.path.isdir(d) and 'bad' not in str(d):
        conds = (
            lambda x: 'GaugeModel_logs' in (str(x)),
            lambda x: 'HMC_' not in str(x),
            lambda x: Path(x).is_dir(),
            lambda x: 'bad' not in (str(x)),
            lambda x: 'old' not in (str(x)),
            lambda x: os.path.isdir(os.path.join(str(x), 'run_data')),
            lambda x: os.path.isfile(os.path.join(str(x), 'run_params.z')),
        )
        l2hmc_dirs += _look(d, 'L16_b', conds)

        console.log(f'len(l2hmc_dirs): {len(l2hmc_dirs)}')
        console.log(f'len(unique(l2hmc_dirs)): {len(np.unique(l2hmc_dirs))}')

In [13]:
len(np.unique(l2hmc_dirs))

81

In [14]:
# 51 (2021-02-25, 10:43:01) 
# 57 (2021-02-25, 21:21:16)
# 63 (2021-02-25, 02:17:39)
# 69 (2021-02-26, 08:12:44)
# 97 (2021-02-27, 23:05:10)
console.log(len(l2hmc_dirs))
l2hmc_dirs = list(np.unique(l2hmc_dirs))
console.log(len(l2hmc_dirs))
# 41 (2021-02-25, 10:43:01)
# 44 (2021-02-25, 21:21:16)
# 47 (2021-02-25, 02:17:39)
# 50 (2021-02-26, 08:12:44)
# 78 (2021-02-27, 23:05:10)

In [15]:
_l2hmc_log_dirs = []
for ldir in l2hmc_dirs:
    lstr = str(ldir)
    _l2hmc_log_dirs.append(lstr[:lstr.find('j/')+1])

_l2hmc_log_dirs = list(np.unique(_l2hmc_log_dirs))
len(_l2hmc_log_dirs)

#l2hmc_log_dirs_file = os.path.join(PROJECT_DIR, 'l2hmc_log_dirs_list.txt')
#with open(l2hmc_log_dirs_file, 'w') as f:
#    for ldir in l2hmc_dirs:
#        f.write(str(ldir) + '\n')

26

In [16]:
from copy import deepcopy
import utils.autocorr as acr

hdirs_bad = []
hdata = {b: {} for b in np.arange(1, 8)}
for idx, hdir in enumerate(hmc_dirs):
    #if 'b512' not in str(hdir):
    #    continue
        
    console.log(f'({idx} / {len(hmc_dirs)}) hdir: {hdir}')
    tint_files = [x for x in Path(hdir).rglob('*tau_int_data.z*') if x.is_file()]
    tint_files += [x for x in Path(hdir).rglob('*tint_data.z*') if x.is_file()]
    for f in tint_files:
        loaded = io.loadz(f)
        
        if 'tau_int_data' in str(f):
            run_dir = os.path.dirname(os.path.split(f)[0])
        else:
            run_dir, _ = os.path.split(f)
            
        try:
            params = acr._get_important_params(run_dir)
        except (FileNotFoundError, ValueError) as err:
            console.log(f'err: {err}, skipping!', style=RED)
            hdirs_bad.append(hdir)
            continue
            
        tint = loaded.get('tint', loaded.get('tau_int', None))
        narr = loaded.get('narr', loaded.get('draws', None))
        
        if tint is not None and narr is not None:
            data = deepcopy(params)
            data.update({
                'tint': tint,
                'narr': narr,
            })

            beta = params['beta']
            traj_len = params['traj_len']
            hdata[beta][traj_len] = data

In [17]:
import joblib
from copy import deepcopy

ldirs_bad = []
ldata = {b: {} for b in np.arange(1, 8)}

for idx, ldir in enumerate(l2hmc_dirs):
    if 'bad' in str(ldir):
        continue
    #console.log(f'({idx}/{len(l2hmc_dirs)}) ldir: {ldir}')
    tint_files = [x for x in Path(ldir).rglob('*tint_data.z*') if x.is_file()]
    tint_files += [x for x in Path(ldir).rglob('*tau_int_data.z*') if x.is_file()]
    for f in tint_files:
        try:
            loaded = io.loadz(f)
        except ValueError:
            try:
                with open(f, 'rb') as ff:
                    loaded = joblib.load(ff)
            except ValueError:
                console.rule()
                console.log(f'Unable to load!!!!', style=RED)
                console.log(f'tint_file: {f}', style=RED)
                console.rule()
        
        if 'tau_int_data' in str(f):
            run_dir = os.path.dirname(os.path.split(f)[0])
        else:
            run_dir, _ = os.path.split(f)
            
        try:
            params = acr._get_important_params(run_dir)
        except (FileNotFoundError, ValueError) as err:
            console.log(f'err: {err}, skipping!', style=YELLOW)
            ldirs_bad.append(ldir)
            continue
            
        tint = loaded.get('tint', loaded.get('tau_int', None))
        narr = loaded.get('narr', loaded.get('draws', None))
        
        if tint is not None and narr is not None:
            data = deepcopy(params)
            astr, (bi, bf) = get_annealing_schedule(str(ldir))
            beta = params['beta']
            traj_len = params['traj_len']
            
            if 'bad' in str(run_dir):
                continue

            if bf != beta:
                console.log(run_dir, style='#ffff00')
                continue

            # ----------------------------------------- 
            #if beta == 5.:
            #    if np.abs(traj_len - 1.94) < 0.1 or np.abs(traj_len - 1.55) < 0.01:
            #        continue

            # if beta == 6.:
            #     c1 = np.abs(traj_len - 1.8413677) < 0.001
            #     c2 = np.abs(traj_len - 1.7582482) < 0.001
            #     c3 = np.abs(traj_len - 1.7748458) < 0.001
            #     c4 = np.abs(traj_len - 1.7845551) < 0.001
            #     c5 = np.abs(traj_len - 1.6606886) < 0.001
            #     if c1 or c2 or c3 or c4 or c5: # or c6:
            #         continue

            # if beta == 4. and np.abs(traj_len - 1.73) <= 0.01:
            #     continue

            # if beta == 3.:
            #     if np.abs((traj_len - 1.85)) < 0.01:
            #         continue
            
            # ----------------------------------------- 

            if beta == 6.:
                if bi == 5.:
                    bi = 4.

            if beta == 7.:
                bi = 4.
                bf = 7.

            bi = int(bi)
            bf = int(bf)
            data.update({
                'tint': tint,
                'narr': narr,
                'beta_init': bi,
                'beta_final': bf,
                'run_dir': ldir,
                'lf': params['run_params']['num_steps'],
            })
            
            beta = params['beta']
            traj_len = params['traj_len']
            ldata[beta][traj_len] = data

In [18]:
console.log(len(hmc_dirs))
console.log(len(l2hmc_dirs))

In [19]:
from config import PROJECT_DIR
ldirs_txt_file = os.path.join(PROJECT_DIR, 'l2hmc_dirs_list.txt')

ldx = 0
with open(ldirs_txt_file, 'w') as f:
    for ldir in l2hmc_dirs:
        if 'bad' not in str(ldir):
            _ = f.write(f'{ldir}\n')
            ldx += 1
            
hdx = 0
hdirs_txt_file = os.path.join(PROJECT_DIR, 'hmc_dirs_list.txt')
with open(hdirs_txt_file, 'w') as f:
    for hdir in hmc_dirs:
        if 'bad' not in str(hdir):
            _ = f.write(f'{hdir}\n')
            hdx += 1
#np.savetxt(ldirs_txt_file, np.array(l2hmc_dirs))#np.array([str(i) for i in l2hmc_dirs if 'bad' not in str(i)]))
console.log(f'ldx: {ldx}')
console.log(f'hdx: {hdx}')

In [20]:
len(np.unique(l2hmc_dirs))

81

In [21]:
inf_cfgs0f = os.path.join(l2hmc_dirs[0], 'inference_configs.json')
with open(inf_cfgs0f, 'rt') as f:
    inf_cfgs0 = json.load(f)
    
console.log(inf_cfgs0)

In [22]:
rp0 = io.loadz(os.path.join(l2hmc_dirs[0], 'run_params.z'))
console.log(rp0)

In [23]:
def bootstrap(x, *, nboot, binsize):
    boots = []
    x = x.reshape(-1, binsize, *x.shape[1:])
    for i in range(nboot):
        boots.append(np.mean(x[np.random.randint(len(x), size=len(x))], axis=(0,1)))
    return np.mean(boots), np.std(boots)

In [24]:
from utils.data_utils import therm_arr

def calc_susceptibility(log_dir, therm_frac=0.5, nboot=1000, binsize=64, verbose=True, charges_dict=None):
    if 'L8' in str(log_dir):
        return None, None
    
    run_params = io.loadz(os.path.join(log_dir, 'run_params.z'))
    beta = run_params.get('beta', None)
    
    charges_file = os.path.join(log_dir, 'run_data', 'charges.z')
    charges = io.loadz(charges_file)
    charges, _ = therm_arr(charges, therm_frac)
    avg, err = bootstrap(charges ** 2, nboot=nboot, binsize=binsize)
    
    if 'L16' in str(log_dir):
        volume = 16 * 16
        
    #elif 'L8' in str(log_dir):
        
    avg /= volume
    err /= volume
    if verbose:
        console.log(fr'log_dir: {log_dir}')
        console.log(fr'beta: {beta}, < Q ** 2 > = {avg:.6g} +/- {err:.6g}', style=YELLOW)
    if charges_dict is not None:
        charges_dict[beta]['log_dir'].append(log_dir)
        charges_dict[beta]['charges_file'].append(charges_file)
        charges_dict[beta]['avg'].append(avg)
        charges_dict[beta]['err'].append(err)
        
    return (avg, err)

In [25]:
betas = np.arange(1, 8, 1)
charges_dict = {
    b: {
        'avg': [],
        'err': [],
        'charges_file': [],
        'log_dir': [],
    } for b in betas
}
console.log(charges_dict)

In [28]:
for ldir in sorted(l2hmc_dirs):
    _ = calc_susceptibility(ldir, therm_frac=0.5, nboot=1000, binsize=100, charges_dict=charges_dict, verbose=True)

In [ ]:
!touch /lus/grand/projects/DLHMC/l2hmc-qcd/top_susceptibility/top_susceptibility.json

In [29]:
tstr = io.get_timestamp('%Y-%m-%d')
suscept_dir = os.path.join(PROJECT_DIR, f'top_susceptibility_{tstr}')
io.check_else_make_dir(suscept_dir)

suscept_file = os.path.join(suscept_dir, 'top_susceptibility.z')

io.savez(charges_dict, suscept_file, name='top_susceptibility')

In [30]:
json_file = os.path.join(suscept_dir, 'top_susceptibility.json')
_charges_dict = {
    str(k): v for k, v in charges_dict.items()
}
with open(json_file, 'wt') as f:
    json.dump(_charges_dict, f, indent=4, ensure_ascii=False, cls=io.NumpyEncoder)

In [39]:
suscept_avg = []
suscept_err = []
for key, val in charges_dict.items():
    avg = val['avg']
    err = val['err']
    suscept_avg.append(np.mean(avg))
    suscept_err.append(np.mean(err))
    print(f'beta: {key}, avg: {np.mean(avg)} +/- {np.mean(err)}')
    
suscept_avg = np.array(suscept_avg)
suscept_err = np.array(suscept_err)
console.log(suscept_avg.shape)
console.log(suscept_avg[0].shape)

beta: 1, avg: 0.04062659852206707 +/- 0.0011388790444470942
beta: 2, avg: 0.01936093345284462 +/- 0.0006120987236499786
beta: 3, avg: 0.014073810659540005 +/- 0.006516212210930788
beta: 4, avg: 0.007326309472167243 +/- 0.0009346037283345746
beta: 5, avg: 0.005990119454892058 +/- 0.0012575404409864476
beta: 6, avg: 0.004627215010779244 +/- 0.0014645063877521483
beta: 7, avg: 0.0038475130632933644 +/- 0.0006012286224480098


In [45]:
from config import PROJECT_DIR

fig, ax = plt.subplots(constrained_layout=True, figsize=(4, 3))

betas = np.arange(1, 8)
_ = ax.errorbar(betas, suscept_avg, yerr=suscept_err, ls='--')
_ = ax.set_xlabel(r'$\beta$')
_ = ax.set_ylabel(r'$\chi_{\mathcal{Q}}$')
_ = fig.savefig(os.path.join(PROJECT_DIR, 'plots', 'susceptibility_plot.pdf'), dpi=400, bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
console.log(charges_dict)

In [ ]:
q0 = calc_susceptibility(l2hmc_dirs[0], therm_frac=0.2)

In [ ]:
q0_avg, q0_err = bootstrap(q0**2, nboot=100, binsize=16)
console.log(f'q_avg: {q0_avg:.5g} +/- {q0_err:.5g}')
#X_mean, X_err = bootstrap(Q**2, Nboot=100, binsize=16)

In [ ]:
betas = np.arange(1, 8)
for beta in betas:
    num_entries = len(list(ldata[beta].keys()))
    console.rule(f'beta: {beta}, {num_entries} entries')
    #console.log(ldata[beta])

## mplta

## New heading

In [68]:
from copy import deepcopy
from cycler import cycler
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from config import PLOTS_DIR, MARKERS
from utils.plotting_utils import truncate_colormap, set_size
from config import COLORS, MARKERS, NetWeights
import mpltex

betas = np.arange(2, 8)

data_l2hmc = {b: {} for b in betas}
bad_data_l2hmc = {b: {} for b in betas}

cmap = plt.get_cmap('viridis_r')
color = cmap(0.1)
colors = 10 * ['C0', 'C1', cmap(0.3), 'C2', 'deeppink', 'C3', 'C4', 'C6', 'C7', cmap(0.7), 'C8', 'C3']
markers = 10 * ['s', 'd', 'v', '^', '<', '>', 'p', '*', 'x', '+', 'H', 'D', 'X', 'P']
styles = {
    2.: {
        2.: {'color': 'deeppink', 'marker': 's'},
    },
    3.: {
        2.: {'color': 'deeppink', 'marker': 's'},
        3.: {'color': 'C0', 'marker': 'd'},
    },
    4.: {
        2.: {'color': 'deeppink', 'marker': 's'},
        3.: [{'color': 'C0', 'marker': 'd'}, {'color': 'C1', 'marker': 'v'}],
        4.: {'color': 'C2', 'marker': '^'}
    },
    5.: {
        4.: {'color': 'deeppink', 'marker': 's'},
        5.: {'color': 'C0', 'marker': 'd'}
    },
}

datestr = io.get_timestamp('%Y-%m-%d')
timestr = io.get_timestamp('%H%M')
outdir = os.path.join(PLOTS_DIR, f'autocorrs_{datestr}')
io.check_else_make_dir(outdir)

sns.set_context('talk')
#sns.set_palette('bright')
#sns.set_palette("Set2")
#sns.set_palette('tab10')
#sns.set_palette('Set1')

#colors = 10 * ['deeppink', 'C0', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8', 'C9']
#sns.set_palette(sns.color_palette('flare', n_colors=6))#, as_cmap=True))
sns.set_palette('bright', color_codes=True)

#colors = 10 * ['deeppink', 'C0', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8', 'C9']

#sns.set_style('whitegrid')
sns.set_style('ticks')
plt.rc('text', usetex=True)
#sns.set_style("ticks", {"xtick.major.size": 0, "xtick.minor.size": 0, "ytick.major.size": 0, "ytick.minor.size": 0})


betas = np.arange(2, 8)
titles = {
    #1.: r"$\beta = 1$",
    2.: r"$\beta = 2$",
    3.: r"$\beta = 3$",
    4.: r"$\beta = 4$",
    5.: r"$\beta = 5$",
    6.: r"$\beta = 6$",
    7.: r"$\beta = 7$",
}


from mpltex.aps import _params
from mpltex import MPLdecorator

#latex_preamble = r"\usepackage{siunitx}\sisetup{detect-all}\usepackage{helvet}\usepackage[eulergreek,EULERGREEK]{sansmath}\sansmath"
# _params.update({
#    'savefig.format': 'eps',
#    'text.usetex': True,
#    'text.latex.preamble': (            # LaTeX preamble
#        r'\usepackage{lmodern}'
#        r'\usepackage{amsmath}'
#        r'\usepackage{amssymb}'
#        r'\usepackage{microtype}'
#        r'\usepackage{bm}'
#        r'\usepackage{bbm}'
#    ),
#})
#pdf_decorator = MPLdecorator(_params)

#@pdf_decorator
def combined_plots(figsize=(12, 4), betas=np.arange(2, 8), hthin=1, lthin=1, halpha=1., lalpha=1.):
    ncols = len(betas)
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=figsize, sharey='row', constrained_layout=True)
    axes = axes.flatten()

    axdict = dict(zip(betas, axes))

    tint_tlen_hmc = {b: {}  for b in betas}

    tint_tlen_l2hmc = {b: {} for b in betas}
    l2hmc_data = {b: {} for b in betas}
    hmc_data = {b: {} for b in betas}
    for beta in betas:
        if beta == 1.:
            continue

        htdata = hdata[beta]
        ax = axdict[beta]

        htlens = []
        htdata = dict(sorted(hdata[beta].items()))
        cmap = truncate_colormap('Greys', minval=0.2, maxval=1., n=len(list(htdata.keys())))
        sns.set_palette('bright')
        
        linestyles = mpltex.linestyles(
            lines=[],
            markers=['s', 'v', '^', 'D', '<', '>', 'p', 'h'], 
            colors=['C0', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8'],
            hollow_styles=[True, False]
            #hollow_styles=[True]
        )
        linestyles_hmc = mpltex.linestyles(lines=[], markers=['o'], hollow_styles=[False], colors=[])
        
        for idx, (traj_len, ht) in enumerate(htdata.items()):
            lf = ht['lf']
            x = ht['narr']
            y = lf * ht['tint']
            yavg = np.mean(y, axis=-1)
            yerr = np.std(y, axis=-1) / np.log(10)
            #yvar = np.var(y, axis=-1) / np.log(10)
            #upper, lower = np.percentile(np.log10(y), [5, 95], axis=-1)

            if beta < 5.:
                keep = np.where(x < 5e4)
                x = x[keep]
                y = y[keep]
                yavg = yavg[keep]
                yerr = yerr[keep]

            keep = np.isfinite(yavg)
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]

            htlens.append(traj_len)

            if beta == 7.:
                keep = np.squeeze(np.where(yavg > 1000))
                x = x[keep]
                y = y[keep]
                yavg = yavg[keep]
                yerr = yerr[keep]

            color = cmap(traj_len / 4.)
            label = None
            if idx == 0:
                label = r'$\lambda = $' + f'{traj_len:6.1f}'

            try:
                x_ = x[::int(hthin)]
                y_ = yavg[::int(hthin)]
                yerr_ = yerr[::int(hthin)]
                _ = ax.errorbar(x_, y_, yerr_, color=color, marker='o', alpha=halpha)#, mew=1, alpha=0.5)
                #_ = ax.errorbar(x[::thin], yavg[::thin], yerr=yerr[::thin],
                #                color=color, marker='o', mew=1, alpha=0.5)
                hmc_data[beta][traj_len] = {
                    'narr': x, 'tint': y,
                    'tint_avg': yavg, 'tint_err': yerr,
                    'tint_best_avg': yavg[-1], 'tint_best_err': yerr[-1],
                    'color': color, 'marker': '.', 'label': label,
                    'params': ht.get('run_params', None),
                }
            except IndexError:
                continue

        idx = 0
        ltdata = dict(sorted(ldata[beta].items(), reverse=False, key=lambda x: x[1]['beta_init']))
        for traj_len, tint in ltdata.items():
            traj_len = float(traj_len)
            if traj_len < 1.:
                continue

            label=r'$\lambda = $' +  f'{traj_len:6.02f}'
            x = tint['narr']
            y = tint['tint'] * tint['lf']
            y = y[-x.shape[0]:]
            yavg = np.mean(y, -1)
            yerr = np.std(y, -1) / np.log(10)

            keep = np.where(x > 1000)
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]

            data_l2hmc[beta].update({traj_len: deepcopy(tint)})

            color = colors[idx]
            marker = markers[idx]

            try:
                run_dir = tint['run_params']['run_dir']

                if 'bad' in str(run_dir):
                    continue

                astr, (bi, bf) = get_annealing_schedule(run_dir)

            except KeyError:
                continue

            if bf != beta:
                console.log(run_dir, style='#ffff00')
                continue

            if beta == 5.:
                if np.abs(traj_len - 1.94) < 0.1 or np.abs(traj_len - 1.55) < 0.01:
                    continue

            if beta == 6.:
                c1 = np.abs(traj_len - 1.8413677) < 0.001
                c2 = np.abs(traj_len - 1.7582482) < 0.001
                c3 = np.abs(traj_len - 1.7748458) < 0.001
                c4 = np.abs(traj_len - 1.7845551) < 0.001
                c5 = np.abs(traj_len - 1.6606886) < 0.001
                c6 = np.abs(traj_len - 2.0876097) < 0.001
                if c1 or c2 or c3 or c4 or c5 or c6:
                    continue

            if beta == 4.:
                keep = np.where(x < 1e5)
                x = x[keep]
                y = y[keep]
                yavg = yavg[keep]
                yerr = yerr[keep]

            if beta == 4. and np.abs(traj_len - 1.73) <= 0.01:
                continue

            if beta == 3.:
                if np.abs((traj_len - 1.85)) < 0.01:
                    continue
                    
            tlen_str = str(traj_len)

            if beta < 5.:
                train_steps = 1e5

            if beta == 5.:
                c1 = tlen_str.startswith('1.7049')
                c2 = tlen_str.startswith('1.73549')
                c3 = tlen_str.startswith('1.737')
                c4 = tlen_str.startswith('1.74395')
                if c1 or c2 or c3 or c4:
                    train_steps = 5e5
                    bi = 1.
                    bf = 5.
                if tlen_str.startswith('1.937'):
                    bi = 4.
                    bf = 5.
                    train_steps = 1e5
                if tlen_str.startswith('1.546'):
                    train_steps = 5e5
                    bi = 4.
                    bf = 5.
                if tlen_str.startswith('1.4977'):
                    train_steps = 5e5
                    bi = 3.
                    bf = 5.

            if beta == 6.:
                if tlen_str.startswith('1.886'):
                    train_steps = 5e5
                    bi = 1.
                    bf = 6.
                if tlen_str.startswith('1.917'):
                    bi = 1.
                    bf = 6.
                    train_steps = 5e5
                c1 = tlen_str.startswith('2.094')
                c2 = tlen_str.startswith('2.087')
                c3 = tlen_str.startswith('2.085')
                c4 = tlen_str.startswith('2.093')
                if c1 or c2 or c3 or c4:
                    train_steps = 3e5
                    bi = 3.5
                    bf = 6.

                c5 = tlen_str.startswith('1.8189')
                c6 = tlen_str.startswith('1.8295')
                c7 = tlen_str.startswith('1.6606')
                if c5 or c6 or c7:
                    train_steps = 7.5e5
                    bi = 4.
                    bf = 6.

                if tlen_str.startswith('1.400'):
                    bi = 2.
                    bf = 6.
                    train_steps = 5e5

            if beta == 7.:

                c1 = tlen_str.startswith('1.998')
                c2 = tlen_str.startswith('1.975')
                c3 = tlen_str.startswith('2.00845')
                c4 = tlen_str.startswith('2.008') 
                c5 = tlen_str.startswith('1.9935') 
                c6 = tlen_str.startswith('1.99235')
                c7 = tlen_str.startswith('2.9400')
                if c1 or c2 or c3 or c4 or c5 or c6 or c7:
                    train_steps = 5e5
                    bi = 1.
                    bf = 7.
                if tlen_str.startswith('1.81'):
                    bi = 4.
                    bf = 7.
                    train_steps = 1e6

            bi = int(bi)
            bf = int(bf)
            tint.update({
                'beta_init': bi,
                'beta_final': bf,
            })

            tint_tlen_l2hmc[beta].update({
                traj_len: {
                    'mc_steps': x,
                    'tint_avg': yavg,
                    'tint_err': yerr,
                    'x': x,
                    'y': y,
                    'yavg': yavg,
                    'yerr': yerr,
                    'label': label,
                    'marker': '.',
                    'color': color,
                    'yerr': yerr,
                    'params': ht.get('run_params', None),
                }
            })

            try:
                #if label is not None:
                x_ = x[::int(lthin)]
                y_ = yavg[::int(lthin)]
                yerr_ = yerr[::int(lthin)]
                _ = ax.errorbar(x_, y_,  yerr=yerr_, alpha=lalpha, **next(linestyles))
                #try:
                #    _ = ax.errorbar(x[::2], yavg[::2], yerr=yerr[::2], **next(linestyles))#ls='', marker=marker, color=color, label=label)#, markersize=3.5)
                #except IndexError:
                #    continue

                # ------------
                # TODO: Fix below for updated labels
                #astr = r"$\beta:$" + f' {bi}' + r"$\rightarrow$" + f'{bf}'
                #@zstr = r"$N_{\mathrm{train}} = $" + f' {int(train_steps):4g}'
                #label = ', '.join([label, astr, zstr])
                # ------------
                
                #else:
                #    _ = ax.errorbar(x[::2], yavg[::2], yerr=yerr[::2], ls='', marker=marker, color=color, markersize=3.5)
                #label=r'$\lambda = $' +  f'{traj_len:6.02f}'
                net_weights = tint['run_params']['net_weights']
                if net_weights != NetWeights(1., 1., 1., 1., 1., 1.):
                    label += f', nw: {[int(i) for i in net_weights]}'
                    zorder = 10

                l2hmc_data[beta][traj_len] = {
                    'narr': x,
                    'tint': y,
                    'tint_avg': yavg,
                    'tint_err': yerr,
                    'tint_best_avg': yavg[-1],
                    'tint_best_err': yerr[-1],
                    'beta_init': bi,
                    #'train_steps': train_steps,
                    'beta_final': bf,
                    #'label_split': label.split(','),
                    'color': color,
                    'marker': marker,
                    'label': label,
                    'params': tint.get('run_params', None),
                }

                _ = ax.set_title(titles[beta])
                idx += 1

            except IndexError:
                continue

        _ = ax.set_title(titles[beta])
        #_ = ax.legend(loc='best', fontsize='x-small', markerscale=0.9)
        _ = ax.set_yscale('log')
        _ = ax.set_xscale('log')
        _ = ax.grid(True, alpha=0.4)
        _ = ax.grid(True, which='minor', alpha=0.1)
        _ = ax.set_xlim((1e3, ax.get_xlim()[-1]))
        #_ = ax.set_ylim((10, ax.get_ylim()[-1]))

    #_ = axdict[5.].set_xlim((1e3, axdict[5.].get_xlim()[-1]))
    _ = axdict[betas[0]].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
    #_ = axdict[5.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
    #_ = axdict[5.].set_xlabel(f'MC Step', loc='left')
    #plt.tight_layout()
    normalize = mcolors.Normalize(vmin=0.5, vmax=4.)
    scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
    _ = scalarmappable.set_array(np.unique(htlens))
    ticks = []
    for b, val in hmc_data.items():
        for tlen, d in val.items():
            ticks.append(tlen)

    ticks = list(np.unique(ticks))
    #colorbar = plt.colorbar(scalarmappable, ax=list(axdict.values()), label=r"$\lambda$ (HMC)", location='right', pad=0.01)#, aspect=40, pad=0.01, location='right')#ticks=ticks, extend='max', location='right')
    #fig.align_labels(axs=list(axdict.values()) + [bigax] + [colorbar])
    
    bigax = fig.add_subplot(111, frameon=False)
    _ = bigax.grid(False)
    bigax.zorder = -100
    _ = bigax.set_xticks([])
    _ = bigax.set_yticks([])
    _ = bigax.set_xticklabels([])
    _ = bigax.set_yticklabels([])
    # hide tick and tick label of the big axis
    _ = bigax.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
    _ = bigax.set_xlabel(r'MC Step')#, labelpad=10.0)
    #_ = bigax.set_xlabel(r'MC Step', labelpad=0.6)
    #_ = plt.xlabel('MC Step')
    #_ = plt.xlabel(r'MC Step')
    colorbar = plt.colorbar(scalarmappable, ax=list(axdict.values()), label=r"$\lambda$ (HMC)", location='right')#, pad=0.01)#, aspect=40, pad=0.01, location='right')#ticks=ticks, extend='max', location='right')
    #_ = fig.execute_constrained_layout()
    #_ = fig.set_constrained_layout_pads()
    _ = plt.xlabel(r'MC Step')
    
    _ = fig.set_constrained_layout_pads()
    _ = fig.align_labels(axs=list(axdict.values()) + [bigax])
    
    mdir = os.path.join(outdir, 'autocorr_vs_mc_step')
    io.check_else_make_dir(mdir)

    outfile = os.path.join(mdir, f'autocorr_vs_mc_step_{timestr}')#'.pdf')
    console.log(f'Saving figure to: {outfile}.')
    _ = plt.savefig(outfile, bbox_inches='tight')
    plt.show()
    
    outputs = {
        'hmc': hmc_data,
        'l2hmc': l2hmc_data,
    }
    
    return outputs, (fig, axdict)

In [69]:
outputs, _ = combined_plots(figsize=(12, 5.), betas=np.arange(2, 8), hthin=1, lthin=3, halpha=0.6, lalpha=0.75)

hmc_data = outputs['hmc']
l2hmc_data = outputs['l2hmc']

<ipython-input-68-9870aa16d4a2>:94: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=figsize, sharey='row', constrained_layout=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
console.log(charges_dict)

In [ ]:
console.log(charges_dict)

In [ ]:
#for tlen, x in _l2hmc_data[6.].items():
#    console.log(f'tlen: {tlen}, run_dir: {x["params"]["run_dir"]}')
    

In [ ]:
#import tikzplotlib
#W = 5.8
plt.style.use('default')
sns.set_context('talk')
sns.set_style('whitegrid')
sns.set_palette('bright')

plt.rcParams.update({
    'font.family': 'lmodern',
#    #'font.size': 9.6,  # 9.6000000000000000001
#    #'axes.labelsize': 9.6,
#    #'legend.fontsize': 8.8,
#    #'legend.labelspacing': 0.5, # 0.5
#    #'legend.handletextpad': 0.8,  # 0.8
    'text.usetex': True,
    'text.latex.preamble': (            # LaTeX preamble
        r'\usepackage{lmodern}'
        r'\usepackage{amsmath}'
        r'\usepackage{amssymb}'
        r'\usepackage{microtype}'
        r'\usepackage{bm}'
        r'\usepackage{bbm}'
    )
})

In [75]:
#console = Console(theme=Theme({"repr.number": "#ff79ff", "repr.path": '#f92672'}), width=240, log_time=True, log_time_format='[%X]')

from copy import deepcopy

l2hmc_data_ = {}
for beta, val in ldata.items():
    l2hmc_data_[beta] = {}
    console.rule(f'{beta}')
    for tlen, data in val.items():
        #if tlen < 1.:
        #    continue
        #if beta == 7. and tlen > 2.:
        #    continue
            
        run_dir = data['run_params']['run_dir']
        tint_avg = np.mean(data['tint'][-1, :])
        tint_err = np.std(data['tint'][-1, :])
        bi = data.get('beta_init', None)
        bf = data.get('beta_final', None)
        if bi is not None:
            bi = int(bi)
            
        if bf is not None:
            bf = int(bf)
        
        l2hmc_data_[beta][tlen] = deepcopy(data)
        l2hmc_data_[beta][tlen].update({
            'beta_init': bi,
            'beta_final': bf,
            'tint_avg': tint_avg,
            'tint_err': tint_err,
        })
        
        log_str = ', '.join([x for x in [f'{beta}', f'{bi}', f'{bf}', f'{tlen}',
                                         f'{tint_avg}', f'{tint_err}']])
        console.log(log_str, style=GREEN)
        #log_str = (
        #    f'beta: {beta}, bi: {bi}, bf: {bf}, '
        #    f'traj_len: {tlen}, tint[-1]: '
        #    f'{tint_avg} +/- {tint_err}'
        #)
        #
        #, style=GREEN)
        
        console.log(f' run_dir: {run_dir}', style=GREEN)
        console.log(80*'-', style=RED)

In [ ]:
l2hmc_data_[6.].pop(1.82951420545578)

In [ ]:
extra_ = l2hmc_data[6.].pop(1.82951420545578)

In [ ]:
htlens_arr = np.array(htlens_arr)
htlens_arr = np.unique(htlens_arr[htlens_arr < 4.])
htlens_arr

In [ ]:
_ = l2hmc_data[7.].pop(1.9935019)
_ = l2hmc_data[7.].pop(1.9984337)

In [55]:
with open(os.path.join(PROJECT_DIR, f'output{timestr}.txt'), 'w') as f:
    _l2hmc_data = {}
    for beta, val in l2hmc_data.items():
        f.write('\n\n')
        f.write('\n' + 60 * '=' + f'  beta: {beta}, entries: {len(list(val.keys()))}  ' + 60 * '=' + '\n')
        f.write('\n')
        ##console.rule(f'beta: {beta}, # entries: {len(list(val.keys()))}')
        _l2hmc_data[int(beta)] = {}
        for tlen, data in val.items():
            rd = data['params']['run_dir']
            #ld = data['params']['log_dir']
            #console.log(f'log_dir: {ld}')
            tlen_ = np.round(tlen, decimals=7)
            f.write('\n')
            h1 = f'beta: {beta}, lambda: {tlen}'
            #run_dir: {rd}' # + '\n'
            underline = '-' * len(h1)
            f.write('\n'.join([h1, underline]))
            f.write('\n')
            f.write(f'  run_dir: {rd}\n\n')
            f.write('\n')
            #f.write(h1 )
            #f.write('\n')
            #f.write('\n' + '- - - - - - - - - - - - - - - - - - - - - - - ' + '\n')
            #console.log(f'beta: {beta}, lambda: {tlen_}\n run_dir: {rd}')
            #console.log(' ')
        
            if tlen_ in _l2hmc_data[beta]:
                console.log(f'existing tlen: {tlen_}', style=RED)

            #console.log(data.keys())
            _l2hmc_data[beta][str(tlen_)] = deepcopy(data)
            
            y = data['tint'] # * data['params']['num_steps'] #data['lf']
            yavg = np.mean(y, -1)
            yerr = np.std(y, -1) / np.log(10)
            
            _l2hmc_data[beta][str(tlen_)].update({
                'tint_avg': yavg,
                'tint_err': yerr,
                #'params': data['run_params'],
                'color': data['color'],
                'label': data['label'],
                'marker': data['marker'],
            })
            #_l2hmc_data[beta][str(tlen_)].update({'lf': data['params']['num_steps']})

In [56]:
with open(os.path.join(PROJECT_DIR, f'output_hmc{timestr}.txt'), 'w') as f:
    _hmc_data = {}
    for beta, val in hmc_data.items():
        f.write('\n\n')
        f.write('\n' + 60 * '=' + f'  beta: {beta}, entries: {len(list(val.keys()))}  ' + 60 * '=' + '\n')
        f.write('\n')
        ##console.rule(f'beta: {beta}, # entries: {len(list(val.keys()))}')
        _hmc_data[int(beta)] = {}
        for tlen, data in val.items():
            rd = data['params']['run_dir']
            #ld = data['params']['log_dir']
            #console.log(f'log_dir: {ld}')
            tlen_ = np.round(tlen, decimals=7)
            f.write('\n')
            h1 = f'beta: {beta}, lambda: {tlen}'
            #run_dir: {rd}' # + '\n'
            underline = '-' * len(h1)
            f.write('\n'.join([h1, underline]))
            f.write('\n')
            f.write(f'  run_dir: {rd}\n\n')
            f.write('\n')
            #f.write(h1 )
            #f.write('\n')
            #f.write('\n' + '- - - - - - - - - - - - - - - - - - - - - - - ' + '\n')
            #console.log(f'beta: {beta}, lambda: {tlen_}\n run_dir: {rd}')
            #console.log(' ')
        
            if tlen_ in _l2hmc_data[beta]:
                console.log(f'existing tlen: {tlen_}', style=RED)

            #console.log(data.keys())
            _hmc_data[beta][str(tlen_)] = deepcopy(data)
            
            y = data['tint'] # * data['params']['num_steps'] #data['lf']
            yavg = np.mean(y, -1)
            yerr = np.std(y, -1) / np.log(10)
            
            _hmc_data[beta][str(tlen_)].update({
                'tint_avg': yavg,
                'tint_err': yerr,
                #'params': data['run_params'],
                'color': data['color'],
                'label': data['label'],
                'marker': data['marker'],
            })
            #_l2hmc_data[beta][str(tlen_)].update({'lf': data['params']['num_steps']})

In [ ]:
os.listdir(data['run_params']['run_dir'])
inf_configs = io.loadz(os.path.join(data['run_params']['run_dir'], 'inference_configs.z'))
bidx = inf_configs['log_dir'].find('l2hmc-qcd')
base_dir = inf_configs['log_dir'][:bidx]
train_configs_file = os.path.join(base_dir, 'train_configs.json')
with open(train_configs_file, 'rt') as f:
    train_configs = dict(json.load(f))
    
loaded_dir = os.path.abspath(train_configs['dynamics_config']['log_dir'])
    
console.log(loaded_dir)
console.log(train_configs)
#console.log(os.listdir(base_dir))
#console.log(dict(inf_configs))

In [ ]:
extra61 = _l2hmc_data[6.].pop('2.09424', None)
extra61 = _l2hmc_data[6.].pop('2.08532', None)

extra71 = _l2hmc_data[7.].pop('1.9935', None)
extra72 = _l2hmc_data[7.].pop('1.99236', None)
#extra73 = _l2hmc_data[7.].pop('1.99236')

extra71 = _l2hmc_data[7.].pop('1.994', None)

In [71]:
def plot_autocorrs_for_beta(hmc_data, l2hmc_data, beta, legend=True, figsize=(4, 3), labelsize=None, cmap=None):
    fig, axes = plt.subplots(ncols=int(2), sharey=True, figsize=figsize, constrained_layout=True, )
    ax, ax1 = axes
    for idx, (tlen, data) in enumerate(hmc_data[beta].items()):
        x = data['narr']
        y = data['tint_avg']
        yerr = data['tint_err']
        #label = data['label']
        if cmap is None:
            cmap = truncate_colormap('Greys', minval=0.2, maxval=1., n=len(list(hmc_data[beta].keys())))
        
        
        run_dir = data['params']['run_dir']
        
        label = None
        if 'nw' in run_dir:
            nwidx = run_dir.find('nw')
            nw_str = run_dir[nwidx:nwidx+8]
            label = nw_str
           
        
        if beta < 5.:
            keep = np.where(x < 1.05e4)
        elif beta == 5.:
            keep = np.where(x < 5e4)
        else:
            keep = np.where(x < 5e5)

        
        _ = ax.errorbar(x, y, yerr=yerr, label=label,# markersize=3.5, capsize=2.5,
                        ls='', marker='o', alpha=0.7, color=cmap(tlen/3.75))
        _ = ax1.errorbar(tlen, y[-1], yerr=yerr[-1],# capsize=2.5,
                        ls='', marker='o', alpha=0.7, color=cmap(tlen/3.75))

    for idx, (tlen, data) in enumerate(dict(sorted(_l2hmc_data[beta].items())).items()):
        tlen = float(tlen)
        x = data['narr']
        y = data['tint_avg']
        yerr = data['tint_err']
        label = data['label']
        #label=r'$\lambda = $' +  f'{traj_len:6.02f}'
        #net_weights = tint['run_params']['net_weights']
        #if net_weights != NetWeights(1., 1., 1., 1., 1., 1.):
        #    label += f', nw: {[int(i) for i in net_weights]}'
        #    zorder = 10

        #label = data['label']
        run_dir = data['params']['run_dir']
        
        #label = None
        #if 'nw' in run_dir:
        #    nwidx = run_dir.find('nw')
        #    nw_str = run_dir[nwidx:nwidx+8]
        #    label = nw_str
           
        _ = ax.errorbar(x, y, yerr=yerr, label=label, color=data['color'],
                        ls='', marker=data['marker'])# markersize=3.5, capsize=2.5)#, marker=marker, color=color)
        _ = ax1.errorbar(tlen, y[-1], yerr=yerr[-1], label=label, color=data['color'],
                        ls='', marker=data['marker'])# markersize=3.5, capsize=2.5)# marker=MARKERS[idx])

    _ = ax.set_xscale('log')
    _ = ax.set_yscale('log')
    _ = ax1.set_yscale('log')
    if labelsize is not None:
        _ = ax.tick_params(labelsize=labelsize)
        _ = ax1.tick_params(labelsize=labelsize)
        _ = fig.suptitle(titles[beta], fontsize=labelsize)
    else:
        _ = fig.suptitle(titles[beta])
        
    #_ = ax.set_ylim((100, ax.get_ylim()[-1]))
    if legend:
        _ = ax1.legend(bbox_to_anchor=(0, 1.03), loc='upper left', fontsize=7.)#, markerscale=0.8)
        _ = plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
    _ = ax.grid(True, which='major', alpha=0.4)
    _ = ax.grid(True, which='minor', alpha=0.1)
    _ = ax1.grid(True, which='major', alpha=0.4)
    _ = ax1.grid(True, which='minor', alpha=0.1)

    _ = ax.set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
    _ = ax.set_xlabel(f'MC Step')
    _ = ax1.set_xlabel(r"$\lambda$")

    
    #normalize = mcolors.Normalize(vmin=0.25, vmax=3.75)
    #scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
    #_ = scalarmappable.set_array(np.unique(htlens))
    #colorbar = plt.colorbar(scalarmappable, label=r"$\lambda$ (HMC)", ax=axes, location='right', aspect=40, pad=0.001)        
    
  
    
    # Place a legend to the right of this smaller subplot.

    
    #big_axes = fig.add_subplot(111, frame_on=False)
    #_ = plt.tick_params(labelcolor='none', bottom=False, left=False)
    #_ = big_axes.grid(False)
    #_ = big_axes.set_xlabel(f'MC Step')
    #_ = fig.align_labels(axs=list(axdict.values()) + [big_axes])
    

    mcdir = os.path.join(outdir, 'autocorr_vs_mc_step')
    mcbdir = os.path.join(mcdir, f'betas_{timestr}')
    io.check_else_make_dir(mcbdir)
    outfile = os.path.join(mcbdir, '.'.join([f'autocorr_vs_mc_step_beta{int(beta)}'.rstrip('.'), 'pdf']))
    console.log(f'Saving figure to: {outfile}')
    _ = plt.savefig(outfile, bbox_inches='tight')
        
    return fig, ax

In [58]:
tstr = io.get_timestamp('%H%M')
lfile = os.path.join(outdir, f'l2hmc_data_{tstr}.z')
io.savez(_l2hmc_data, lfile, name=f'l2hmc_data_{tstr}')

In [59]:
hfile = os.path.join(outdir, f'hmc_data_{tstr}.z')
io.savez(_hmc_data, hfile, name=f'hmc_data_{tstr}')

In [ ]:
console.log(_l2hmc_data[3.].keys())

In [ ]:
_ = _l2hmc_data[3.].pop('2.0924108', None)

In [ ]:
train_steps = {
    2.: 1e5,
    3.: 1e5,
    4.: 1e5,
    5.: 5e5,
    6.: 6e5
}
ax.tick_params('labelsize': 16.)

In [75]:
import matplotlib as mpl

sns.set_context('paper', font_scale=1.5)
with mpl.rc_context({
    #'font.family': 'lmodern',
    #'figure.figsize': (6, 3),
    'figure.figsize': (6, 4),
    #'font.size': 12.,  # 9.6000000000000000001
    #'axes.labelsize': 16.,
    #'legend.fontsize': 8.,
    #'legend.labelspacing': 0.2, # 0.5
    #'legend.handletextpad': 0.3,  # 0.8
    'text.usetex': True,
    #'text.latex.preamble': (            # LaTeX preamble
    #    #r'\usepackage{lmodern}'
    #    r'\usepackage{amsmath}'
    #    r'\usepackage{amssymb}'
    #    r'\usepackage{microtype}'
    #    r'\usepackage{bm}'
    #    r'\usepackage{bbm}'
    #)
}):
    timestr = io.get_timestamp('%H')
    fig, ax = plot_autocorrs_for_beta(hmc_data, _l2hmc_data, 2., figsize=(6, 4), legend=False)#, labelsize=16.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, _l2hmc_data, 3., figsize=(6, 4), legend=False)#, labelsize=16.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, _l2hmc_data, 4., figsize=(6, 4), legend=False)#, labelsize=16.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, _l2hmc_data, 5., figsize=(6, 4), legend=False)#, labelsize=16.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, _l2hmc_data, 6., figsize=(6, 4), legend=False)#, labelsize=16.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, _l2hmc_data, 7., figsize=(6, 4), legend=False)#, labelsize=16.)

<ipython-input-71-2bca7a02d95f>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(ncols=int(2), sharey=True, figsize=figsize, constrained_layout=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-71-2bca7a02d95f>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(ncols=int(2), sharey=True, figsize=figsize, constrained_layout=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-71-2bca7a02d95f>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(ncols=int(2), sharey=True, figsize=figsize, constrained_layout=True)


...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-71-2bca7a02d95f>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(ncols=int(2), sharey=True, figsize=figsize, constrained_layout=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-71-2bca7a02d95f>:2: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(ncols=int(2), sharey=True, figsize=figsize, constrained_layout=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
_l2hmc_data[6.].keys()

In [ ]:
bright_cmap = sns.color_palette(as_cmap=True)
bright_cmap

In [ ]:
import matplotlib as mpl

sns.set_context('paper')
with mpl.rc_context({
    'font.family': 'lmodern',
    'figure.figsize': (4, 3),
    'font.size': 12.,  # 9.6000000000000000001
    'axes.labelsize': 12.,
    'legend.fontsize': 8.,
    'legend.labelspacing': 0.2, # 0.5
    'legend.handletextpad': 0.3,  # 0.8
    'text.usetex': True,
    'text.latex.preamble': (            # LaTeX preamble
        r'\usepackage{lmodern}'
        r'\usepackage{amsmath}'
        r'\usepackage{amssymb}'
        r'\usepackage{microtype}'
        r'\usepackage{bm}'
        r'\usepackage{bbm}'
    )
}):
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 2.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 3.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 4.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 5.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 6.)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 7.)
   

In [ ]:
_ = _l2hmc_data[]

In [ ]:
sns.set_context('paper')
with mpl.rc_context({
    'font.family': 'lmodern',
    'figure.figsize': (4, 3),
    'font.size': 12.,  # 9.6000000000000000001
    'axes.labelsize': 12.,
    'legend.fontsize': 8.,
    'legend.labelspacing': 0.2, # 0.5
    'legend.handletextpad': 0.3,  # 0.8
    'text.usetex': True,
    'text.latex.preamble': (            # LaTeX preamble
        r'\usepackage{lmodern}'
        r'\usepackage{amsmath}'
        r'\usepackage{amssymb}'
        r'\usepackage{microtype}'
        r'\usepackage{bm}'
        r'\usepackage{bbm}'
    )
}):
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 7.)
   

In [ ]:
def plot_autocorrs_for_beta(hmc_data, l2hmc_data, beta, legend=True, capsize=0.):
    fig, axes = plt.subplots(ncols=int(2), sharey=True, figsize=(8, 3), constrained_layout=True)
    ax, ax1 = axes
    for idx, (tlen, data) in enumerate(hmc_data[beta].items()):
        x = data['narr']
        y = data['tint_avg']
        yerr = data['tint_err']
        label = data['label']
        
        if beta < 5.:
            keep = np.where(x < 1.05e4)
        elif beta == 5.:
            keep = np.where(x < 5e4)
        else:
            keep = np.where(x < 5e5)

        
        _ = ax.errorbar(x, y, yerr=yerr, label=label,# markersize=3.5, capsize=capsize,
                        ls='', marker='o', alpha=0.7, color=cmap(tlen/3.75))
        _ = ax1.errorbar(tlen, y[-1], yerr=yerr[-1],# markersize=3.5, capsize=capsize,
                        ls='', marker='o', alpha=0.7, color=cmap(tlen/3.75))

    for idx, (tlen, data) in enumerate(_l2hmc_data[beta].items()):
        tlen = float(tlen)
        x = data['narr']
        y = data['tint_avg']
        yerr = data['tint_err']
        label = data['label']
        _ = ax.errorbar(x, y, yerr=yerr, label=label, color=data['color'],
                        ls='', marker=data['marker'])#,# markersize=3.5, capsize=capsize)#, marker=marker, color=color)
        _ = ax1.errorbar(tlen, y[-1], yerr=yerr[-1], label=label, color=data['color'],
                        ls='', marker=data['marker'])#, markersize=3.5, capsize=capsize)# marker=MARKERS[idx])

    _ = fig.suptitle(titles[beta])
    _ = ax.set_xscale('log')
    _ = ax.set_yscale('log')
    _ = ax1.set_yscale('log')
    #_ = ax.set_ylim((100, ax.get_ylim()[-1]))
    if legend:
        _ = ax1.legend(loc='best')#, fontsize=7.)#, markerscale=0.8)
        
    _ = ax.grid(True, which='major', alpha=0.4)
    _ = ax.grid(True, which='minor', alpha=0.1)
    _ = ax1.grid(True, which='major', alpha=0.4)
    _ = ax1.grid(True, which='minor', alpha=0.1)

    _ = ax.set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
    _ = ax.set_xlabel(f'MC Step')
    _ = ax1.set_xlabel(r"$\lambda$")

    #normalize = mcolors.Normalize(vmin=0.25, vmax=3.75)
    #scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
    #_ = scalarmappable.set_array(np.unique(htlens))
    #colorbar = plt.colorbar(scalarmappable, label=r"$\lambda$ (HMC)", ax=axes, location='right', aspect=40, pad=0.001)        

    mcdir = os.path.join(outdir, 'autocorr_vs_mc_step')
    mcbdir = os.path.join(mcdir, f'betas_{timestr}')
    io.check_else_make_dir(mcbdir)
    outfile = os.path.join(mcbdir, '.'.join([f'autocorr_vs_mc_step_beta{int(beta)}'.rstrip('.'), 'pdf']))
    console.log(f'Saving figure to: {outfile}')
    _ = plt.savefig(outfile, bbox_inches='tight')
        
    return fig, ax

In [ ]:
sns.set_context('paper')
with mpl.rc_context({
    #'font.family': 'lmodern',
    'figure.figsize': (4, 3),
    'font.size': 12.,  # 9.6000000000000000001
    'axes.labelsize': 12.,
    'legend.fontsize': 8.,
    'legend.labelspacing': 0.2, # 0.5
    'legend.handletextpad': 0.3,  # 0.8
    'text.usetex': True,
    'text.latex.preamble': (            # LaTeX preamble
        #r'\usepackage{lmodern}'
        r'\usepackage{amsmath}'
        r'\usepackage{amssymb}'
        r'\usepackage{microtype}'
        r'\usepackage{bm}'
        r'\usepackage{bbm}'
    )
}):
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 2., legend=False)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 3., legend=False)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 4., legend=False)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 5., legend=False)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 6., legend=False)
    fig, ax = plot_autocorrs_for_beta(hmc_data, l2hmc_data, 7., legend=False)

In [ ]:
#def _plot_autocoors_from_data(data, beta)
import matplotlib as mpl

with mpl.rc_context({
    #'font.family': 'lmodern',
    'font.size': 12.,  # 9.6000000000000000001
    'axes.labelsize': 12.,
    'legend.fontsize': 8.,
    'legend.labelspacing': 0.2, # 0.5
    'legend.handletextpad': 0.3,  # 0.8
    'text.usetex': True,
    #'text.usetex': True,
    'text.latex.preamble': (            # LaTeX preamble
        #r'\usepackage{lmodern}'
        r'\usepackage{amsmath}'
        r'\usepackage{amssymb}'
        r'\usepackage{microtype}'
        r'\usepackage{bm}'
        r'\usepackage{bbm}'
    )
}):
    timestr = io.get_timestamp('%H%M')

    for jdx, beta in enumerate(betas):
        fig, ax = plot_autocorrs_for_beta(l2hmc_data, hmc_data, beta)
        #fig, axes = plt.subplots(ncols=2, sharey=True, figsize=(8, 3), constrained_layout=True)
        #ax, ax1 = axes


New heading

In [ ]:
axes.shape

In [ ]:
axdict = {}
for idx, beta in enumerate(betas):
    axdict[beta] = axes[idx, :]

In [ ]:
klen(l2hmc_dirs)

In [ ]:
l2hmc_dirs


In [ ]:
rdl_dict = {
    str(b): [] for b in betas
}
for beta, val in l2hmc_data.items():
    for tlen, data in val.items():
        rdl_dict[str(beta)].append(data['params']['run_dir'])
        
rdh_dict = {
    str(b): [] for b in betas
}
for beta, val in hmc_data.items():
    for tlen, data in val.items():
        rdh_dict[str(beta)].append(data['params']['run_dir'])

In [ ]:
rd_hmc_file = os.path.join(PROJECT_DIR, 'run_dirs_hmc.json')
with open(rd_hmc_file, 'w') as f:
    json.dump(rdh_dict, f, indent=4)

In [ ]:
rd_l2hmc_file = os.path.join(PROJECT_DIR, 'run_dirs_l2hmc.json')
with open(rd_l2hmc_file, 'w') as f:
    json.dump(rdl_dict, f, indent=4)

In [ ]:
from config import PROJECT_DIR
#$rdl_file = os.path.join(PROJECT_DIR, 'l2hmc_run_dirs_thetaGPU.txt')
io.save_dict(rdl_dict, out_dir=PROJECT_DIR, name='l2hmc_run_dirs_thetaGPU')
#console.log(rdl_dict)

In [ ]:
!tlmgr update --all

In [76]:
import matplotlib as mpl
from copy import deepcopy


#markers = 10 * ['s', 'd', 'v', '^', '<', '>', 'p', '*', 'x', '+']
markers = {
    2.: 's',
    3.: '^',
    4.: 'd',
    5.: 'v',
    7.: '*',
    6.: '<', 
}


with mpl.rc_context({
    'font.family': 'lmodern',
    'font.size': 12.,  # 9.6000000000000000001
    'axes.labelsize': 12.,
    'legend.fontsize': 8.,
    'legend.labelspacing': 0.2, # 0.5
    'legend.handletextpad': 0.3,  # 0.8
    'text.usetex': True,
    'text.latex.preamble': (            # LaTeX preamble
        #r'\usepackage[utf-8]{inputenc}'
        #r'\usepackage{lmodern}'
        r'\usepackage{microtype}'
        r'\usepackage{amsmath}'
        r'\usepackage{amssymb}'
        r'\usepackage{microtype}'
        r'\usepackage{bm}'
        r'\usepackage{bbm}'
    )
}):
    W = 5.8
    timestr = io.get_timestamp('%H%M')

    #fig, axes = plt.subplots(nrows=2, ncols=3,# figsize=(4, 3),
    #                         sharey='row', sharex='col', constrained_layout=True)
    try:
        htlens_arr = np.unique(sorted(np.array(htlens_arr)[np.array(htlens_arr) < 4.]))
        cmap = truncate_colormap('binary', n=len(htlens_arr), minval=0.12, maxval=1.)
    except NameError:
        pass
        
    fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(12, 3),
                             sharey=True,# sharex=True,
                             #sharey='row', sharex='col',
                             #sharey='row', sharex='col',
                             constrained_layout=True)
    #fig, axes = plt.subplots(nrows=3, ncols=3, constrained_layout=True)
    axes = axes.flatten()

    #cmap = truncate_colormap('binary', minval=0.2, maxval=1., n=num_colors)

    htlens_arr = []
    axdict = dict(zip(betas, axes))
    for beta, ax in axdict.items():
        # ----------------------------------------
        htint = hmc_data[beta]
        
        tlens = list(htint.keys())
        htlens_arr.extend(tlens)
        htint_ = dict(sorted(htint.items(), key=lambda z: z[0]), reverse=True)
        for traj_len, data in htint.items():
            x = data['narr']
            y = data['tint_avg']
            yerr = data['tint_err']
            label = data['label']
            #color = data['color']
            color = cmap(traj_len / 3.75)
            
            if beta < 5.:
                keep = np.where(x < 1.05e4)
            elif beta == 5.:
                keep = np.where(x < 5e4)
            else:
                keep = np.where(x < 5e5)
                
            x = x[keep]
            y = y[keep]
            yerr = yerr[keep]
            
            _ = ax.errorbar(x, y, yerr=yerr, ls='', capsize=2.5,
                            marker='o', color=color, alpha=0.4)
                            #markersize=3.5, alpha=0.7)

        # ----------------------------------------
        ltint = _l2hmc_data[beta]
        ltint_ = dict(sorted(ltint.items(), key=lambda z: z[1]['beta_init']))
        #ltint = dict(sorted(l2hmc_data[beta].items(), key=lambda x: x['bi']))
        #ltdata = dict(sorted(data.items(), reverse=False))
        for idx, (traj_len, data) in enumerate(ltint_.items()):
            traj_len = float(traj_len)
            x = data['narr']
            y = data['tint_avg']
            yerr = data['tint_err']
            label = data['label']
            #marker = data['marker']
            marker = markers[beta]
            color = data['color']
            if beta == 7. and traj_len in [1.9935019, 1.9984337]:
                continue
                
            if beta == 5.:
                keep = np.where(x < 1e4)
                x = x[keep]
                y = y[keep]
                yerr = yerr[keep]
            
            try:
                _ = ax.errorbar(x[::2], y[::2], yerr=yerr[::2], label=label,
                                ls='', marker=marker, color=color, capsize=2.5)
                                #markersize=3.5)#, alpha=0.5)
            except IndexError:
                console.log(f'traj_len: {traj_len}\n data: {data}', style=YELLOW)
                continue


        _ = ax.set_title(titles[beta])
        _ = ax.set_xscale('log')
        _ = ax.set_yscale('log')
        #_ = ax.set_ylim((100, ax.get_ylim()[-1]))
        #_ = ax.legend(loc='best', fontsize=7.)#, markerscale=0.8)
        _ = ax.grid(True, which='major', alpha=0.4)
        _ = ax.grid(True, which='minor', alpha=0.1)

        
    #htlens_arr = np.unique(sorted(np.array(htlens_arr)[np.array(htlens_arr) < 4.]))
    normalize = mcolors.Normalize(vmin=0.25, vmax=3.75)
    scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
    _ = scalarmappable.set_array(np.unique(htlens))
    colorbar = plt.colorbar(scalarmappable, label=r"$\lambda$ (HMC)", ax=list(axdict.values()), location='right', aspect=40, pad=0.001)#, use_gridspec=True)
    
    
    _ = axdict[5.].set_xlim((axdict[5.].get_xlim()[0], 1.25e4))
    _ = axdict[6.].set_xlim((axdict[6.].get_xlim()[0], 1.2e5))
    _ = axdict[7.].set_xlim((axdict[7.].get_xlim()[0], 1.2e5))

    #_ = axdict[5.].set_xlim((1e3, axdict[5.].get_xlim()[-1]))
    _ = axdict[2.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
    #_ = axdict[5.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
    #_ = axdict[5.].set_xlabel(f'MC Step', loc='left')
    #_ = axdict[5.].legend(loc='upper left', fontsize=7.)#, markerscale=0.8)
    #_ = axdict[6.].legend(loc='lower right', fontsize=7.)#, markerscale=0.8)
    #_ = axdict[7.].legend(loc='lower right', fontsize=7.)#, markerscale=0.8)
    #_ = axdict[4.].set_xlabel(f'MC Step', loc='right')
    big_axes = fig.add_subplot(111, frame_on=False)
    _ = plt.tick_params(labelcolor='none', bottom=False, left=False)
    _ = big_axes.grid(False)
    _ = big_axes.set_xlabel(f'MC Step')
    _ = fig.align_labels(axs=list(axdict.values()) + [big_axes])
    
    mcdir = os.path.join(outdir, 'autocorr_vs_mc_step')
    io.check_else_make_dir(mcdir)
    outfile = os.path.join(mcdir, f'autocorr_vs_mc_step_{timestr}.pdf')
    console.log(f'Saving figure to: {outfile}.')
    _ = plt.savefig(outfile, dpi=400, bbox_inches='tight')

<ipython-input-76-3dcc9995b6d3>:46: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrows=1, ncols=6, figsize=(12, 3),


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

NameError: name 'htlens' is not defined

In [ ]:
console.log(_l2hmc_data[5.].keys())

In [ ]:
l2hmc_dirs[0]

In [ ]:
#tlens_ = ['1.992', '1.998', '1.975', '2.008']
tlens_ = ['1.99843', '2.00846', '2.00872']#, '1.975', '2.008']
for tlen in tlens_:
    bi = 1
    bf = 7
    tlenf = float(tlen)
    tlen_str = r'$\lambda = $' + f'{tlenf:6.02f}'
    beta_str = r"$\beta : $ " + f' {bi}' + r"$\rightarrow$" + f'{bf}'
    label = ', '.join([tlen_str, beta_str])
    _l2hmc_data[7.][tlen].update({
        'beta_init': bi,
        'beta_final': bf,
        'label': label,
    })

In [ ]:
for tlen, val in _l2hmc_data[5.].items():
    bi = val['beta_init']
    bf = val['beta_final']
    run_dir = val['params']['run_dir']
    console.log(f'tlen: {tlen}, bi: {bi}, bf: {bf}')
    console.log(f'run_dir: {run_dir}')

In [ ]:
plt.tick_params()

In [80]:
from utils.plotting_utils import truncate_colormap

sns.set_context('talk')
#sns.set_palette('bright')
#sns.set_palette("Set2")
#sns.set_palette('tab10')
#sns.set_palette('Set1')

#colors = 10 * ['deeppink', 'C0', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8', 'C9']
#sns.set_palette(sns.color_palette('flare', n_colors=6))#, as_cmap=True))
sns.set_palette('bright', color_codes=True)
sns.set_style('ticks', )
plt.rc('text', usetex=False)
#plt.rc('')
#plt.tick_params(direction='in')

def plot_autocorr_vs_traj_len(
        figsize=(12, 4),
        betas=np.arange(2, 8),
        halpha=0.7,
        lalpha=0.9
):
    with mpl.rc_context({
        #'font.family': 'lmodern',
        #'font.size': 12,  # 9.6000000000000000001
        #'axes.labelsize': 12.,
        #'legend.fontsize': 8,
        #'legend.labelspacing': 0.2, # 0.5
        #'legend.handletextpad': 0.3,  # 0.8
        'text.usetex': True,
        'text.latex.preamble': (            # LaTeX preamble
            #r'\usepackage{lmodern}'
            r'\usepackage{amsmath}'
            r'\usepackage{amssymb}'
            r'\usepackage{microtype}'
            r'\usepackage{bm}'
            r'\usepackage{bbm}'
        )
    }):
        plt.tick_params(direction='in')
        timestr = io.get_timestamp('%H%M')
        fig, axes = plt.subplots(nrows=1, ncols=len(betas), figsize=figsize,
                                 sharey=True,# sharex=True,
                                 constrained_layout=True)
        axes = axes.flatten()
        axdict = dict(zip(betas, axes))
        for beta, ax in axdict.items():
            # ----------------------------------------
            linestyles = mpltex.linestyles(
                lines=[],
                markers=['s', 'v', '^', 'D', '<', '>', 'p', 'h'], 
                colors=['C0', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8'],
                hollow_styles=[True, False]
                #hollow_styles=[True]
            )
            linestyles_hmc = mpltex.linestyles(lines=[], markers=['o'], hollow_styles=[False], colors=[])
            htint = hmc_data[beta]
            htint_ = dict(sorted(htint.items(), key=lambda z: z[0]), reverse=True)
            hmc_cmap = truncate_colormap('Greys', minval=0.2, maxval=1.0, n=len(list(htint.keys())))
            for traj_len, data in htint.items():
                if traj_len > 4.:
                    continue

                x = data['narr']
                y = data['tint_avg']
                yerr = data['tint_err']
                label = data['label']
                color = hmc_cmap(traj_len / 3.75)
                #color = data['color']

                keep = np.where(x <= 1e5)
                x = x[keep]
                y = y[keep]
                yerr = yerr[keep]

                _ = ax.errorbar(traj_len, y[-1], yerr=yerr[-1], ls='',
                                marker='o', color=color, alpha=0.7)#, capsize=2.5)
                                #markersize=3.5, alpha=0.7)

            # ----------------------------------------
            ltint = _l2hmc_data[beta]
            ltint_ = dict(sorted(ltint.items(), key=lambda z: z[1]['beta_init']))
            #ltint = dict(sorted(l2hmc_data[beta].items(), key=lambda x: x['bi']))
            #ltdata = dict(sorted(data.items(), reverse=False))
            for idx, (traj_len, data) in enumerate(ltint_.items()):
                traj_len = float(traj_len)
                x = data['narr']
                y = data['tint_avg']
                yerr = data['tint_err']
                label = data['label']
                marker = data['marker']
                color = data['color']
                bi = int(data['beta_init'])
                bf = int(data['beta_final'])
                label = r"$\beta:$ " + f'{bi}' + r"$\rightarrow$" + f'{bf}'

                _ = ax.errorbar(traj_len, y[-1], yerr=yerr[-1], label=label, **next(linestyles))
                                #ls='', marker=marker, color=color)#, capsize=2.5)
                                #markersize=3.5)#, alpha=0.5)

            _ = ax.set_title(titles[beta])
            #_ = ax.set_xscale('log')
            _ = ax.set_yscale('log')
            #_ = ax.set_ylim((100, ax.get_ylim()[-1]))
            #_ = ax.legend(loc='best', fontsize=8.)#, markerscale=0.8)
            _ = ax.grid(True, which='major', alpha=0.4)
            _ = ax.grid(True, which='minor', alpha=0.1)

        #normalize = mcolors.Normalize(vmin=0.25, vmax=3.75)
        #scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
        #_ = scalarmappable.set_array(np.unique(htlens))
        #colorbar = plt.colorbar(scalarmappable, label=r"$\lambda$ (HMC)", ax=axes, location='right', aspect=40, pad=0.001)#, use_gridspec=True)

        #_ = axdict[5.].set_xlim((1e3, axdict[5.].get_xlim()[-1]))
        _ = axdict[betas[0]].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
        #_ = axdict[5.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
        #_ = axdict[4.].set_xlabel(r"$\lambda$", loc='right')
        #_ = axdict[5.].legend(loc='lower right', fontsize=8.)#, markerscale=0.8)
        #_ = axdict[6.].legend(loc='lower right', fontsize=8.)#, markerscale=0.8)
        #_ = axdict[7.].legend(loc='lower right', fontsize=8.)#, markerscale=0.8)

        #big_axes = fig.add_subplot(111, frame_on=False)
        #_ = plt.tick_params(labelcolor='none', bottom=False, left=False)
        #_ = big_axes.grid(False)
        #_ = big_axes.set_xlabel(r"$\lambda$")
        #_ = big_axes.set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
        #_ = axdict[4.].set_xlabel(r"$\lambda$", loc='right')
        for beta, ax in axdict.items():
            _ = ax.set_xticks([1, 2, 3])
            _ = ax.set_xticklabels([r"$1$", r"$2$", r"$3$"])
            #if beta == betas[0] or beta == betas[-1]:
            #    continue
                
            #_ = ax.set_yticks([])
            #_ = ax.set_yticklabels([])
                
            #_ = ax.set_xticklabels(['', r"$2$", ''])


        big_axes = fig.add_subplot(111, frame_on=False)
        _ = plt.tick_params(labelcolor='none', bottom=False, left=False)
        _ = big_axes.grid(False)
        _ = big_axes.set_xlabel(r"$\lambda$")
        _ = fig.align_labels(axs=list(axes) + [big_axes])
        
        #for beta, ax in axdict.items():
        #    if beta == betas[0]:
        #        continue
        #    else:
        #        _ = ax.set_yticks([])
        #        _ = ax.set_yticklabels([])
            #_ = ax.set_xticks([1, 2, 3])
            #_ = ax.set_xticklabels([r"$1$", r"$2$", r"$3$"])
            #if beta == betas[0] or beta == betas[-1]:
            #    continue
                
                


        tlen_dir = os.path.join(outdir, 'autocorr_vs_traj_len')
        io.check_else_make_dir(tlen_dir)
        outfile = os.path.join(tlen_dir, f'autocorr_vs_traj_len_{timestr}.pdf')
        console.log(f'Saving figure to: {outfile}.')
        _ = plt.savefig(outfile, dpi=400, bbox_inches='tight')
        #_ = tikzplotlib.save(outfile.rstrip('.pdf') + '.tex')

In [81]:
plot_autocorr_vs_traj_len(figsize=(9, 3), betas=np.arange(2, 8))

<ipython-input-80-1d1c5922548b>:42: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrows=1, ncols=len(betas), figsize=figsize,


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [105]:
betas

array([2, 3, 4, 5, 6, 7])

In [ ]:
!touch /lus/grand/projects/DLHMC/l2hmc-qcd/top_susceptibility/top_susceptibility.json

In [ ]:
timestr

In [ ]:
ddir = os.path.join(outdir, 'data')
io.check_else_make_dir(ddir)
hmc_data_file = os.path.join(ddir, f'hmc_data_{timestr}.z')
l2hmc_data_file = os.path.join(ddir, f'l2hmc_data_{timestr}.z')
io.savez(hmc_data, hmc_data_file)
io.savez(l2hmc_data, l2hmc_data_file)

In [86]:
import matplotlib as mpl
from copy import deepcopy

plt.style.use('default')
#sns.set_context('paper')
sns.set_style('ticks')
sns.set_palette('bright')
cmap = plt.get_cmap('viridis_r')
#color = cmap(0.75)
color = '#1AC938'
#color = cmap(0.25)
#color = '#0182CA'
#plt.rc_context()
#default_rcParams = deepcopy(plt.rcParams)
#plt.rcParams.update(default_rcParams)
#plt.rcParams['text.usetex'] = True
sns.set_context('talk')

#plt.rcParams.update({
#    'text.usetex': True,
#})
sns.set_palette('bright', color_codes=True)
sns.set_style('whitegrid')
plt.rc('text', usetex=False)
sns.set_style('ticks')


with mpl.rc_context({
    #'font.family': 'lmodern',
    #'font.size': 12,  # 9.6000000000000000001
    #'axes.labelsize': 12.,
    #'legend.fontsize': 8,
    #'legend.labelspacing': 0.2, # 0.5
    #'legend.handletextpad': 0.3,  # 0.8
    'text.usetex': True,
    'text.latex.preamble': (            # LaTeX preamble
        #r'\usepackage{lmodern}'
        r'\usepackage{amsmath}'
        r'\usepackage{amssymb}'
        r'\usepackage{microtype}'
        r'\usepackage{bm}'
        r'\usepackage{bbm}'
    )
}):
    W = 5.8
    timestr = io.get_timestamp('%H%M')
    fig, ax = plt.subplots(figsize=(9, 3), constrained_layout=True)
    _ = ax.grid(True, which='major', alpha=0.4)
    _ = ax.grid(True, which='minor', alpha=0.2)
    
    hbetas = []
    htint_avg_arr = []
    htint_err_arr = []
    for beta, val in hmc_data.items():
        harr = []
        herr = []
        for traj_len, htdata in val.items():
            harr.append(htdata['tint_avg'][-1])
            herr.append(htdata['tint_err'][-1])
            yerr = htdata['tint_err'][-1]
            y = htdata['tint_avg'][-1]
            #_  = ax.errorbar(beta, y, yerr=yerr, marker='.', color=htdata['color'], ls='')#, label='HMC', lw=2.)

        #betas_ = len(harr) * [beta]
        #_  = ax.errorbar(betas_, harr, yerr=herr, marker='.', color=htdata['color'], ls='')#, label='HMC', lw=2.)
        hbetas.append(beta)
        htint_avg_arr.append(np.mean(harr))
        htint_err_arr.append(np.mean(herr))

    lbetas = []
    ltint_avg_arr = []
    ltint_err_arr = []
    for beta, val in l2hmc_data.items():
        larr = []
        lerr = []
        linestyles = mpltex.linestyles(
            lines=[],
            markers=['s', 'v', '^', 'D', '<', '>', 'p', 'h'], 
            colors=['C0', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8'],
            hollow_styles=[True, False]
            #hollow_styles=[True]
        )
        for traj_len, ltdata in val.items():
            larr.append(ltdata['tint_avg'][-1])
            lerr.append(ltdata['tint_err'][-1])
            yerr = ltdata['tint_err'][-1]
            y = ltdata['tint_avg'][-1]
            #_  = ax.errorbar(beta, y, yerr=yerr, marker=ltdata['marker'], color=ltdata['color'], ls='')#, label=ltdata['label'])#, lw=2.)
            
        betas_ = len(larr) * [beta]
        #_  = ax.errorbar(betas_, larr, yerr=lerr, color=ltdata['color'], marker=ltdata['marker'], ls='')#, label='trained', marker='s', lw=2.)
        lbetas.append(beta)
        ltint_avg_arr.append(np.mean(larr))
        ltint_err_arr.append(np.mean(lerr))
    
    hlabel = r"$\left\langle\mathrm{HMC}\right\rangle$"
    llabel = r"$\left\langle\mathrm{trained}\right\rangle$"
    _  = ax.errorbar(hbetas, htint_avg_arr, yerr=htint_err_arr, marker='o', alpha=1., color='k', ls='--', label=hlabel)#, lw=2., capsize=2.5)
    _  = ax.errorbar(lbetas, ltint_avg_arr, yerr=ltint_err_arr, marker='s', alpha=1., color=color, ls='-', label=llabel)#, lw=2., capsize=2.5)
    #_ = ax.legend(loc='upper left')
    _ = ax.set_yscale('log')
    _ = ax.set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
    _ = ax.legend(loc='best')
    #_ = ax.set_autoscaley_on(True)
    #_ = ax.set_ylim((ax.get_ylim()[0], 1e5))
    #_ = ax.annotate(r"$\left\langle\mathrm{HMC}\right\rangle$", xy=(6.3, 2.5e4), color='k')
    #_ = ax.annotate(r"$\left\langle\mathrm{trained}\right\rangle$", xy=(6.15, 3.2e2), color='deeppink')
    _ = ax.set_xlabel(r"$\beta$")
    #_ = axdict[6.].legend(loc='upper left', fontsize='xx-small', markerscale=0.8)
    _ = ax.grid(True, which='major', alpha=0.4)
    _ = ax.grid(True, which='minor', alpha=0.2)
    plt.grid(True, which='both', )
    #_ = sns.despine(fig=fig, ax=ax, top=True, left=True, right=True, bottom=True, trim=True)

    bdir = os.path.join(outdir, 'autocorr_vs_beta')
    io.check_else_make_dir(bdir)
    outfile = os.path.join(bdir, f'autocorr_vs_beta_{timestr}.pdf')
    console.log(f'Saving figure to: {outfile}.')
    _ = plt.savefig(outfile, dpi=1000, bbox_inches='tight')

<ipython-input-86-a2195936ebb2>:47: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, ax = plt.subplots(figsize=(9, 3), constrained_layout=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
print_l2hmc_data(l2hmc_data)

In [ ]:
import matplotlib as mpl
from copy import deepcopy

#colors = 10 * ['C0', 'C1', 'C2', 'C3', 'C4', 'C6', 'C7', 'C8', 'C9']
#plt.style.use('default')
#sns.set_context('paper')
#sns.set_style('whitegrid')
#sns.set_palette('bright')
#plt.rc_context()
#default_rcParams = deepcopy(plt.rcParams)
#plt.rcParams.update(default_rcParams)
#plt.rcParams['text.usetex'] = True
sns.set_context('paper')

#plt.rcParams.update({
#    'text.usetex': True,
#})

sns.set_palette('bright')
sns.set_context('paper')
cmap = plt.get_cmap('viridis_r')
color = '#0182CA'


with mpl.rc_context({
    'font.family': 'lmodern',
    #'font.size': 12.,  # 9.6000000000000000001
    'figure.figsize': (4, 3),
    #'axes.labelsize': 12.,
    #'legend.fontsize''#0182CA': 8.,
    #'legend.labelspacing': 0.2, # 0.5
    #'legend.handletextpad': 0.3,  # 0.8
    'text.usetex': False,
    'text.latex.preamble': (            # LaTeX preamble
        #r'\usepackage{lmodern}'
        r'\usepackage{amsmath}'
        r'\usepackage{amssymb}'
        r'\usepackage{microtype}'
        r'\usepackage{bm}'
        r'\usepackage{bbm}'
    )
}):
    W = 5.8
    timestr = io.get_timestamp('%H%M')

    #fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(4, 3), # figsize=(6.694444, 4.), # figsize=(W, H),
     #                        sharey='row', sharex='col', constrained_layout=True)
    #fig, axes = plt.subplots(nrows=3, ncols=3, constrained_layout=True)
    #axes = axes.flatten()

    #cmap = truncate_colormap('binary', minval=0.2, maxval=1., n=num_colors)

    #axdict = dict(zip(betas, axes))
    #for beta, ax in axdict.items():
    hdx = 0
    for beta in np.arange(5, 8, 1):
        if beta == 3. or beta == 4.:# or beta == 5. or beta == 6.:
            continue
        #if beta == 2. or beta == 3. or beta == 4.:
        #    continue
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8, 2.5), # figsize=(6.694444, 4.), # figsize=(W, H),
                               sharey='row', sharex='col', constrained_layout=True)
        ax, ax1 = ax.flatten()

        htint = hmc_data[beta]
        for traj_len, data in htint.items():
            if hdx == 3:
                label = 'HMC'
                hdx += 1
            else:
                label = None
            
            _ = ax.errorbar(data['narr'], data['tint_avg'], yerr=data['tint_err'], label=label,
                            ls='', marker='o', color=data['color'], markersize=3)

            _ = ax1.errorbar(traj_len, data['tint_avg'][-1], yerr=data['tint_err'][-1], label=label,
                            ls='', marker='o', color=data['color'], markersize=3)

        ltint = l2hmc_data[beta]
        ltint_ = dict(sorted(ltint.items(), key=lambda z: z[1]['beta_init']))
        #ltint = dict(sorted(l2hmc_data[beta].items(), key=lambda x: x['bi']))
        #ltdata = dict(sorted(data.items(), reverse=False))
        best = np.inf
        best_traj_len = None
        best_data = None
        for traj_len, data in ltint_.items():
            if np.mean(data['tint_avg']) < best:
                best_traj_len = traj_len
                best_data = data
                
        bi = best_data['beta_init']
        bf = best_data['beta_final']
        label = r"$\beta:$ " + f'{int(bi)}' + r"$\rightarrow$" + f'{int(bf)}'
        _ = ax.errorbar(best_data['narr'], best_data['tint_avg'], yerr=best_data['tint_err'], label='Trained', #label=label, 
                        ls='', marker='s', color='deeppink', markersize=3, markeredgewidth=0.4)
        _ = ax1.errorbar(best_traj_len, best_data['tint_avg'][-1], yerr=best_data['tint_err'][-1], label='Trained',# label=label, 
                        ls='', marker='s', color='deeppink', markersize=3, markeredgewidth=0.4)


        #_ = ax.set_title(titles[beta])
        _ = fig.suptitle(titles[beta])
        #_ = ax.set_xscale('log')
        _ = ax.set_yscale('log')
        _ = ax1.set_yscale('log')
        #_ = ax.set_ylim((100, ax.get_ylim()[-1]))
        if hdx == 3:
            _ = ax.legend(loc='best')#fontsize='small', markerscale=0.8)
        _ = ax.grid(True, which='major', alpha=0.4)
        _ = ax.grid(True, which='minor', alpha=0.1)
        _ = ax1.grid(True, which='major', alpha=0.4)
        _ = ax1.grid(True, which='minor', alpha=0.1)

        #normalize = mcolors.Normalize(vmin=1., vmax=4.)
        #scalarmappable = cm.ScalarMappable(norm=normalize, cmap='Greys')
        #_ = scalarmappable.set_array(np.unique(htlens))
        #colorbar = plt.colorbar(scalarmappable, label=r"$\lambda$ (HMC)", ax=list([ax, ax1]))#, use_gridspec=True)

        #_ = axdict[5.].set_xlim((1e3, axdict[5.].get_xlim()[-1]))
        _ = ax.set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
        if beta < 4.:
            _ = ax.set_xlim((1000, 2.5e4))
        else:
            if beta == 5.:
                _ = ax.set_xlim((1000, 25000))
            if beta == 6.:
                _ = ax.set_xlim((1000, 50000))
            if beta == 7.:
                _ = ax.set_xlim((1000, 1e5))
        #_ = ax1.set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
        _ = ax1.set_xlabel(r"$\lambda=N_{\mathrm{LF}}\cdot \varepsilon$")
        _ = ax.set_xlabel(f'MC Step')
        #_ = axdict[6.].legend(loc='upper left', fontsize='xx-small', markerscale=0.8)
        #_ = axdict[7.].legend(loc='upper left', fontsize='xx-small', markerscale=0.8)

        outfile = os.path.join(outdir, f'autocorr_vs_traj_len_{timestr}_b{beta}'.replace('.', '') + '.pdf')
        console.log(f'Saving figure to: {outfile}.')
        _ = plt.savefig(outfile, dpi=1000, bbox_inches='tight')
        #_ = tikzplotlib.save(outfile.rstrip('.pdf') + '.tex')

In [ ]:
%debug

In [ ]:
%debug

In [ ]:
hbetas = []
htint_avg_arr = []
htint_err_arr = []
for beta, val in hmc_data.items():
    harr = []
    herr = []
    for traj_len, htdata in val.items():
        harr.append(htdata['tint_avg'][-1])
        herr.append(htdata['tint_err'][-1])
        
    hbetas.append(beta)
    htint_avg_arr.append(np.mean(harr))
    htint_err_arr.append(np.mean(herr))

lbetas = []
ltint_avg_arr = []
ltint_err_arr = []
for beta, val in l2hmc_data.items():
    larr = []
    lerr = []
    for traj_len, ltdata in val.items():
        larr.append(ltdata['tint_avg'][-1])
        lerr.append(ltdata['tint_err'][-1])
    lbetas.append(beta)
    ltint_avg_arr.append(np.mean(larr))
    ltint_err_arr.append(np.mean(lerr))

In [ ]:
fig, ax = plt.subplots()
_ = ax.errorbar(hbetas, htint_avg_arr, yerr=htint_err_arr)
_ = ax.errorbar(lbetas, ltint_avg_arr, yerr=ltint_err_arr)

In [ ]:
htint_avg_arr, htint_err_arr

In [ ]:
betas

In [ ]:
betas = []
ltint_avg_arr = []
ltint_err_arr = []
for beta, ltdata in val.items():
    betas.append(beta)
    larr = []
    lerr = []
    for traj_len, ltdata in val.items():
        larr.append(ltdata['tint_avg'][-1])
        lerr.append(ltdata['tint_err'][-1])
    ltint_avg_arr.append(np.mean(larr))
    ltint_err_arr.append(np.mean(lerr))

In [ ]:
from copy import deepcopy

plt.style.use('default')
sns.set_context('talk')
sns.set_style('whitegrid')
sns.set_palette('bright')
#default_rcParams = deepcopy(plt.rcParams)
#plt.rcParams.update(default_rcParams)
plt.rcParams['text.usetex'] = True

#plt.rcParams.update({
#    'text.usetex': True,
#})

W = 5.8
timestr = io.get_timestamp('%H%M')

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(W, W/(4/3)),
                         sharey='row', sharex='col', constrained_layout=True)
#fig, axes = plt.subplots(nrows=3, ncols=3, constrained_layout=True)
axes = axes.flatten()

#cmap = truncate_colormap('binary', minval=0.2, maxval=1., n=num_colors)

axdict = dict(zip(betas, axes))
for beta, ax in axdict.items():
    htint = hmc_data[beta]
    for traj_len, data in htint.items():
        _ = ax.errorbar(data['narr'], data['tint_avg'], yerr=data['tint_err'],
                        ls='', marker='.', color=data['color'], markersize=3)
        
    ltint = l2hmc_data[beta]
    ltint_ = dict(sorted(ltint.items(), key=lambda z: z[1]['beta_init']))
    #ltint = dict(sorted(l2hmc_data[beta].items(), key=lambda x: x['bi']))
    #ltdata = dict(sorted(data.items(), reverse=False))
    for traj_len, data in ltint_.items():
        _ = ax.errorbar(data['narr'], data['tint_avg'], yerr=data['tint_err'], label=data['label'], 
                        ls='', marker=data['marker'], color=data['color'], markersize=3)
        
    
    _ = ax.set_title(titles[beta])
    _ = ax.set_xscale('log')
    _ = ax.set_yscale('log')
    #_ = ax.set_ylim((100, ax.get_ylim()[-1]))
    _ = ax.legend(loc='best', fontsize='x-small', markerscale=0.75)
    _ = ax.grid(True, which='major', alpha=0.4)
    _ = ax.grid(True, which='minor', alpha=0.1)
    
normalize = mcolors.Normalize(vmin=1., vmax=4.)
scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
_ = scalarmappable.set_array(np.unique(htlens))
colorbar = plt.colorbar(scalarmappable, label=r"$\lambda$ (HMC)", ax=list(axdict.values()))#, use_gridspec=True)
    
#_ = axdict[5.].set_xlim((1e3, axdict[5.].get_xlim()[-1]))
_ = axdict[2.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
_ = axdict[5.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
_ = axdict[6.].set_xlabel(f'MC Step')
_ = axdict[6.].legend(loc='upper left', fontsize='x-small', markerscale=0.75)
_ = axdict[7.].legend(loc='upper left', fontsize='x-small', markerscale=0.75)

outfile = os.path.join(outdir, f'autocorr_vs_mc_step_{timestr}.pdf')
console.log(f'Saving figure to: {outfile}.')
_ = plt.savefig(outfile, dpi=400, bbox_inches='tight')
#_ = tikzplotlib.save(outfile.rstrip('.pdf') + '.tex')

In [ ]:
%debug

In [ ]:
ldata[6.].keys()

In [ ]:
for key, val in ldata[6.].items():
    console.log(f'key: {key}\n tint: {val["tint"]}', style=GREEN)

In [ ]:
ldata[6.].pop(1.822643131017685)
ldata[6.].pop(1.7845551669597626)

In [98]:
import matplotlib as mpl
from copy import deepcopy
from cycler import cycler
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from config import PLOTS_DIR, MARKERS
from utils.plotting_utils import truncate_colormap, set_size
from config import COLORS, MARKERS

data_l2hmc = {b: {} for b in betas}
bad_data_l2hmc = {b: {} for b in betas}
colors = 10 * ['deeppink', 'C0', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C3']
markers = 10 * ['s', 'd', 'v', '^', '<', '>', 'p']
styles = {
    2.: {
        2.: {'color': 'deeppink', 'marker': 's'},
    },
    3.: {
        2.: {'color': 'deeppink', 'marker': 's'},
        3.: {'color': 'C0', 'marker': 'd'},
    },
    4.: {
        2.: {'color': 'deeppink', 'marker': 's'},
        3.: [{'color': 'C0', 'marker': 'd'}, {'color': 'C1', 'marker': 'v'}],
        4.: {'color': 'C2', 'marker': '^'}
    },
    5.: {
        4.: {'color': 'deeppink', 'marker': 's'},
        5.: {'color': 'C0', 'marker': 'd'}
    },
}

datestr = io.get_timestamp('%Y-%m-%d')
timestr = io.get_timestamp('%H%M')
outdir = os.path.join(PLOTS_DIR, f'autocorrs_{datestr}')
io.check_else_make_dir(outdir)

sns.set_context('paper')
sns.set_palette('bright')
sns.set_style('whitegrid')
plt.rc('text', usetex=True)


betas = np.arange(2, 8)
titles = {
    #1.: r"$\beta = 1$",
    2.: r"$\beta = 2$",
    3.: r"$\beta = 3$",
    4.: r"$\beta = 4$",
    5.: r"$\beta = 5$",
    6.: r"$\beta = 6$",
    7.: r"$\beta = 7$",
}

fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10, 6),
                         sharey='row', sharex='col',
                         constrained_layout=True)
axes = axes.flatten()

axdict = dict(zip(betas, axes))


tint_tlen_hmc = {
   b: {}  for b in betas
}

tint_tlen_l2hmc = {
    b: {} for b in betas
}
for beta, data in deepcopy(ldata).items():
    if beta == 1.:
        continue
        
    ax = axdict[beta]
    
    htlens = []
    htdata = dict(sorted(hdata[beta].items()))
    cmap = truncate_colormap('binary', minval=0.1, maxval=0.9, n=len(list(htdata.keys())))
    for idx, (traj_len, ht) in enumerate(htdata.items()):
        lf = ht['lf']
        x = ht['narr']
        y = lf * ht['tint']
        yavg = np.mean(y, axis=-1)
        yerr = np.std(y, axis=-1) / np.log(10)
        yvar = np.var(y, axis=-1) / np.log(10)
        upper, lower = np.percentile(np.log10(y), [5, 95], axis=-1)
        if np.max(x) > 5e4:
            y = y[:-2]
            x = x[:-2]
            yavg = yavg[:-2]
            yerr = yerr[:-2]
        
        keep = np.isfinite(yavg)
        x = x[keep]
        y = y[keep]
        yavg = yavg[keep]
        yerr = yerr[keep]
        
        if beta != 4.:
            keep = np.where(x <= 1e5)
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]
            
        if beta > 4.:
            keep = np.where(x <= 7e4)
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]
            
        else:
            keep = np.where(x <= 4e4)
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]
            
        
        htlens.append(traj_len)
        
        if beta == 7.:
            keep = np.squeeze(np.where(yavg > 1000))
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]
            
        
        color = cmap(traj_len / 4.)
        label = None
        if idx == 0:
            label = r'$\lambda = $' + f'{traj_len:.3g}'
            label += ' (HMC)'
            
        tint_tlen_hmc[beta].update({
            traj_len: {
                'x': x,
                'y': y,
                'yavg': yavg,
                'yerr': yerr,
                'color': color,
                'marker': '.',
                'label': label,
                'yerr': yerr,
                'params': ht.get('run_params', None),
            }
        })
            
        try:
            _ = ax.errorbar(traj_len, yavg[-1], yerr=yerr[-1], ls='', marker='.', color=color)#, markersize=3.5, color=color)
        except IndexError:
            console.log(f'beta: {beta}, traj_len: {traj_len}', style=YELLOW)
            
    idx = 0
    ltdata = dict(sorted(data.items(), reverse=True))
    for traj_len, tint in ltdata.items():
        if traj_len < 1.:
            continue
            
        label=r'$\lambda = $' + f'{traj_len:<5.3g}'
        try:
            run_dir = tint['run_params']['run_dir']

            if 'bad' in str(run_dir):
                continue

            astr, (bi, bf) = get_annealing_schedule(run_dir)
        except KeyError:
            continue
            

        if bf != beta:
            console.log(run_dir, style='#ffff00')
            continue
            
        x = tint['narr']
        y = tint['tint']
        
        if beta == 5.:
            if np.abs(traj_len - 1.94) < 0.1 or np.abs(traj_len - 1.55) < 0.01:
                continue
            
        #if beta > 5. and np.max(x) < 1e4:
        #    bad_data_l2hmc[beta].update({
        #        traj_len: deepcopy(tint),
        #    })
        #    continue
            
        if beta == 6. and np.isclose(traj_len, 1.841, rtol=0.001):
            bad_data_l2hmc[beta].update({
                traj_len: deepcopy(tint),
            })
            continue
                
        data_l2hmc[beta].update({
            traj_len: deepcopy(tint),
        })
            
        y *= tint['lf']
        y = y[-x.shape[0]:]
        yavg = np.mean(y, -1)
        yerr = np.std(y, -1) / np.log(10)
        upper, lower = np.percentile(np.log10(y), [5, 95], axis=-1)
        y1 = yavg - yerr
        y2 = yavg + yerr
        if beta == 4. and np.abs(traj_len - 1.73) <= 0.01:
            continue
            
        if beta == 3.:
            if np.abs((traj_len - 1.85)) < 0.01:
                continue
        
        if beta != 4.:
            keep = np.where(x <= 1e5)
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]
            
        if beta > 4.:
            keep = np.where(x <= 7e4)
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]
            
        else:
            keep = np.where(x <= 4e4)
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]
            
        if beta == 6.:
            c1 = np.abs(traj_len - 1.8413677) < 0.001
            c2 = np.abs(traj_len - 1.7582482) < 0.001
            c3 = np.abs(traj_len - 1.7748458) < 0.001
            c4 = np.abs(traj_len - 1.7845551) < 0.001
            c5 = np.abs(traj_len - 1.6606886) < 0.001
            #c6 = np.abs(traj_len - 1.8189294) < 0.001
            #c5 = marker == 'v'
            #c6 = marker == '<'
            if c1 or c2 or c3 or c4 or c5: # or c6:
                continue
        
        color = colors[idx]
        marker = markers[idx]
        
        if beta == 6.:
            if bi == 5.:
                bi = 4.
            
        if beta == 7.:
            bi = 4.
            bf = 7.
                
                
        bi = int(bi)
        bf = int(bf)
            
        astr = r"$\beta:$" + f' {bi}' + r"$\rightarrow$" + f'{bf}'
        label = ', '.join([label, astr])
                
        tint_tlen_l2hmc[beta].update({
            traj_len: {
                'x': x,
                'y': y,
                'yavg': yavg,
                'yerr': yerr,
                'label': label,
                'marker': marker,
                'color': color,
                'yerr': yerr,
                'params': ht.get('run_params', None),
            }
        })
        
        _ = ax.errorbar(traj_len, yavg[-1], yerr=yerr[-1], ls='', marker=marker, color=color, label=label)#, markersize=3.5)
        idx += 1
        _ = ax.set_title(titles[beta])
        
    normalize = mcolors.Normalize(vmin=1., vmax=4.)
    scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
    _ = scalarmappable.set_array(np.unique(htlens))
    colorbar = plt.colorbar(scalarmappable, ax=list(axdict.values()), label=r"$\lambda$ (HMC)")
    
    
    _ = ax.set_title(titles[beta])
    #_ = ax.legend(loc='best', fontsize='small', markerscale=0.9)
    _ = ax.set_yscale('log')
    _ = ax.set_xlim((ax.get_xlim()[0], 3.05))
    #_ = ax.set_xscale('log')
    _ = ax.grid(True, alpha=0.4)
    _ = ax.grid(True, which='minor', alpha=0.1)
    
_ = axdict[2.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
_ = axdict[5.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
_ = axdict[6.].set_xlabel(r"$\lambda$")
_ = axdict[6.].legend(loc='center left', fontsize='x-small', markerscale=0.9)
_ = axdict[7.].legend(loc='lower left', fontsize='x-small', markerscale=0.9)

outfile = os.path.join(outdir, f'autocorr_vs_traj_len_{timestr}.pdf')
console.log(f'Saving figure to: {outfile}.')
_ = plt.savefig(outfile, dpi=400, bbox_inches='tight')

<ipython-input-98-3f2549392300>:55: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10, 6),


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10, 6),
                         sharey='row',
                         #sharey='row', sharex='col',
                         constrained_layout=True)
#fig, axes = plt.subplots(nrows=3, ncols=3, constrained_layout=True)
axes = axes.flatten()

#cmap = truncate_colormap('binary', minval=0.2, maxval=1., n=num_colors)

axdict = dict(zip(betas, axes))

for beta, tlen_data in tint_tlen_hmc.items():
    for tlen, data in tlen_data.items():
        marker = data['marker']
        color = data['color']
        label = data['label']
        yavg = data['yavg']
        yerr = data['yerr']
        try:
            _ = ax.errorbar(tlen, yavg[-1], yerr=yerr[-1],
                            ls='', marker=marker, color=color, label=label, markersize=3.5)
        except IndexError:
            continue
        
for beta, tlen_data in tint_tlen_l2hmc.items():
    for tlen, data in tlen_data.items():
        marker = data['marker']
        color = data['color']
        label = data['label']
        yavg = data['yavg']
        yerr = data['yerr']
        try:
            _ = ax.errorbar(tlen, yavg[-1], yerr=yerr[-1],
                            ls='', marker=marker, color=color, label=label, markersize=3.5)
        except IndexError:
            continue

In [ ]:
from copy import deepcopy
from cycler import cycler
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from config import PLOTS_DIR, MARKERS
from utils.plotting_utils import truncate_colormap, set_size
from config import COLORS, MARKERS

data_l2hmc = {b: {} for b in betas}
bad_data_l2hmc = {b: {} for b in betas}
colors = 10 * ['deeppink', 'C0', 'C1', 'C2', 'C4', 'C5', 'C6', 'C7', 'C8', 'C3']
markers = 10 * ['s', 'd', 'v', '^', '<', '>', 'p']

datestr = io.get_timestamp('%Y-%m-%d')
timestr = io.get_timestamp('%H%M')
outdir = os.path.join(PLOTS_DIR, f'autocorrs_{datestr}')
io.check_else_make_dir(outdir)

sns.set_context('paper')
sns.set_palette('bright')
sns.set_style('whitegrid')
plt.rc('text', usetex=True)
#sns.set_style("ticks", {"xtick.major.size": 0, "xtick.minor.size": 0, "ytick.major.size": 0, "ytick.minor.size": 0})


betas = np.arange(2, 8)
titles = {
    #1.: r"$\beta = 1$",
    2.: r"$\beta = 2$",
    3.: r"$\beta = 3$",
    4.: r"$\beta = 4$",
    5.: r"$\beta = 5$",
    6.: r"$\beta = 6$",
    7.: r"$\beta = 7$",
}

#fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8),
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(10, 6),
                         sharey='row',
                         #sharey='row', sharex='col',
                         constrained_layout=True)
#fig, axes = plt.subplots(nrows=3, ncols=3, constrained_layout=True)
axes = axes.flatten()

#cmap = truncate_colormap('binary', minval=0.2, maxval=1., n=num_colors)

axdict = dict(zip(betas, axes))


tint_tlen_hmc = {
   b: {}  for b in betas
}

tint_tlen_l2hmc = {
    b: {} for b in betas
}
for beta, data in deepcopy(ldata).items():
    if beta == 1.:
        continue
        
    #fig, ax = plt.subplots(figsize=set_size(), constrained_layout=True)
    ax = axdict[beta]
    
    htlens = []
    htdata = dict(sorted(hdata[beta].items()))
    cmap = truncate_colormap('binary', minval=0.1, maxval=0.9, n=len(list(htdata.keys())))
    for idx, (traj_len, ht) in enumerate(htdata.items()):
        lf = ht['lf']
        x = ht['narr']
        y = lf * ht['tint']
        yavg = np.mean(y, axis=-1)
        yerr = np.std(y, axis=-1) / np.log(10)
        yvar = np.var(y, axis=-1) / np.log(10)
        upper, lower = np.percentile(np.log10(y), [5, 95], axis=-1)
        
        keep = np.isfinite(yavg)
        x = x[keep]
        y = y[keep]
        yavg = yavg[keep]
        yerr = yerr[keep]
        htlens.append(traj_len)
        try:
            ylast = yavg[-1]
        except IndexError:
            continue
                
        
        if beta == 7.:
            keep = np.squeeze(np.where(yavg > 1000))
            x = x[keep]
            y = y[keep]
            yavg = yavg[keep]
            yerr = yerr[keep]
            
        tint_tlen_hmc[beta].update({
            traj_len: {
                'x': x,
                'y': y,
                'yerr': yerr,
                'params': ht.get('run_params', None),
            }
        })
        
        color = cmap(traj_len / 4.)
        label = None
        if idx == 0:
            label = r'$\lambda = $' + f'{traj_len:.3g}'
            label += ' (HMC)'
            
        try:
            _ = ax.errorbar(traj_len, yavg[-1], yerr=yerr[-1], ls='', marker='.', markersize=3.5, fillstyle='none', color=color)
        except IndexError:
            continue
            
    idx = 0
    ltdata = dict(sorted(data.items(), reverse=True))
    for traj_len, tint in ltdata.items():
        label=r'$\lambda = $' + f'{traj_len:.3g}'
        try:
            run_dir = tint['run_params']['run_dir']

            if 'bad' in str(run_dir):
                continue

            astr, (bi, bf) = get_annealing_schedule(run_dir)
        except KeyError:
            continue
            

        if bf != beta:
            console.log(run_dir, style='#ffff00')
            continue
            
        #label = ', '.join([label, astr])
        #else:
        #    label = None
            
        #for idx in range(100):
        x = tint['narr']
        y = tint['tint']
        #if tint['traj_len'] < 1. or np.min(x) < 500 or (beta==4. and tint['traj_len'] > 2.):
        #    bad_data_l2hmc[beta].update({
        #        traj_len: deepcopy(tint),
        #    })
        #    continue
        if beta == 6.: 
            if np.abs(traj_len - 1.82) < 0.01:
                bi = 3.
                bf = 6.
            elif np.abs(traj_len - 1.83) < 0.01:
                bi = 4.
                bf = 6.
                
            astr = r"$\beta:$" + f' {bi}' + r"$\rightarrow$" + f'{bf}'
        
        #label = ', '.join([label, astr])
        if beta == 5.:
            if np.abs(traj_len - 1.94) < 0.1 or np.abs(traj_len - 1.55) < 0.01:
                continue
            
        if beta > 5. and np.max(x) < 1e4:
            bad_data_l2hmc[beta].update({
                traj_len: deepcopy(tint),
            })
            continue
            
        if beta == 6. and np.isclose(traj_len, 1.841, rtol=0.001):
            bad_data_l2hmc[beta].update({
                traj_len: deepcopy(tint),
            })
            continue
                
            
        data_l2hmc[beta].update({
            traj_len: deepcopy(tint),
        })
            
        y *= tint['lf']
        y = y[-x.shape[0]:]
        yavg = np.mean(y, -1)
        yerr = np.std(y, -1) / np.log(10)
        upper, lower = np.percentile(np.log10(y), [5, 95], axis=-1)
        y1 = yavg - yerr
        y2 = yavg + yerr
        #y1 = 10**(lower)
        #y2 = 10**(upper)
        #y1 = np.min(np.log(y), axis=-1)
        #y2 = np.max(np.log(y), axis=-1)
        
        color = colors[idx]
        marker = markers[idx]
        if beta == 6.:
            if marker == 's':
                bi = 3.
                bf = 6.
            astr = r"$\beta:$" + f' {bi}' + r"$\rightarrow$" + f'{bf}'
        label = ', '.join([label, astr])
        try:
            ylast = yavg[-1]
            yerrlast = yerr[-1]
        except IndexError:
            continue
                
                
        #console.log(f'beta: {beta}, label: {label},\n run_dir: {run_dir}')
        
        try:
            _ = ax.errorbar(traj_len, yavg[-1], yerr=yerr[-1], ls='', marker=marker, color=color, label=label, markersize=3.5)
            idx += 1
        except IndexError:
            continue
        #_ = ax.fill_between(x, y1=y1, y2=y2, color=color, alpha=0.1, zorder=-idx)
        #if beta > 5.:
        #    _ = ax.set_ylim((100, ax.get_ylim()[-1]))
        #for jdx in range(min(16, y.shape[1])):
        #_ = ax.plot(x, y[:], marker=marker, markersize=0.5, ls='', alpha=0.4, color=color)
            
        _ = ax.set_title(titles[beta])
        #_ = ax.set_title(r'$\beta = $' + f'{beta}')
        #_ = ax.set_xscale('log')
        #_ = ax.set_yscale('log')
        
    normalize = mcolors.Normalize(vmin=1., vmax=4.)
    scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
    _ = scalarmappable.set_array(np.unique(htlens))
    #fig.colorbar(im, orientation="horizontal", pad=0.2)

    #colorbar = fig.colorbar(scalarmappable, label=r"$\lambda$ (HMC)")
    colorbar = plt.colorbar(scalarmappable, ax=list(axdict.values()), label=r"$\lambda$ (HMC)")
    
    
    _ = ax.set_title(titles[beta])
    _ = ax.legend(loc='best', fontsize='x-small')
    #_ = ax.set_xlim((900, ax.get_xlim()[-1]))
    _ = ax.set_yscale('log')
    #_ = ax.set_xscale('log')
    #_ = ax.set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
    _ = ax.grid(True, alpha=0.4)
    _ = ax.grid(True, which='minor', alpha=0.1)
    
_ = axdict[2.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
_ = axdict[5.].set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
_ = axdict[6.].set_xlabel(r"$\lambda$")
#_ = axdict[5.].set_xlim((1e3, axdict[5.].get_xlim()[-1]))
#
for beta, ax in axdict.items():
    #_ = ax.yaxis.get_ticklocs(minor=True)
    #_ = ax.minorticks_on()
    #_ = plt.setp(ax.spines.values(), color='lightgray')
    #_ = plt.setp([ax.get_xticklines(), ax.get_yticklines()], color='white')
    if beta > 5.:
            _ = ax.set_ylim((100, ax.get_ylim()[-1]))
    
outfile = os.path.join(outdir, f'autocorr_traj_len.pdf')
console.log(f'Saving figure to: {outfile}.')
_ = plt.savefig(outfile, dpi=400, bbox_inches='tight')

In [ ]:
lf_arr = []
eps_arr = []
hmc_traj_lens = {}
for beta, val in hdata.items():
    hmc_traj_lens[beta] = {}
    console.rule(f'{int(beta)}')
    for tlen, data in val.items():
        params = data['run_params']
        num_steps = params.get('num_steps', None)
        eps = params.get('eps', params.get('eps_avg', None))
        if eps is not None:
            eps = np.mean(eps)
            
        lf_arr.append(num_steps)
        eps_arr.append(eps)
        try:
            hmc_traj_lens[beta][num_steps].update({
                'beta': beta,
                'tlen': tlen,
                'num_steps': num_steps,
                'eps': eps,
            })
        except KeyError:
            hmc_traj_lens[beta].update({
                num_steps: {
                    'beta': beta,
                    'tlen': tlen,
                    'num_steps': num_steps,
                    'eps': eps,
                },
            })
            
        #num_steps = data['run_params']['num_steps']
        #eps = np.mean(data['run_params']['eps'])
        console.log(f'beta: {beta}, tlen: {tlen}, lf: {num_steps}, eps: {eps}')

In [ ]:
lf_arr = np.unique(lf_arr)
eps_arr = np.unique(eps_arr)
console.log(lf_arr)
console.log(eps_arr)

In [ ]:
console.log(dict(sorted(hmc_traj_lens.items())))

In [ ]:
ldata_ = deepcopy(ldata)
for key, val in ldata[5.].items():
    if val['traj_len'] != key:
        ldata_[5.][float(val['traj_len'])] = val

In [ ]:
data_file = os.path.abspath('/Users/saforem2/l2hmc-qcd/plots/tau_int_plots/2021-01-06-1438/data/ldata.z')
data_ = io.loadz(data_file)

In [ ]:
console.log(ldata[5.0])

In [ ]:
data_copy_ = {
    b: {} for b in np.arange(1,8)
}
for beta, val in data_.items():
    for k, v in val.items():
        lf, eps = k
        data_copy_[beta][lf * eps] = {
            'lf': lf,
            'eps': eps,
            'traj_len': traj_len,
            'beta': beta,
            'tint': v['tau_int'],
            'narr': v['draws'],
            'chains': v['chains'],
        }
        

In [ ]:
for beta, val in data_copy_.items():
    for k, v in val.items():
        tlen = k
        if tlen not in ldata[beta]:
            ldata[beta][tlen] = v

In [ ]:
data_copy_[5.0].keys()

In [ ]:
keys_ = list(data_[5.0].keys())
console.log(data_[5.0][keys_[0]])

In [ ]:
for beta, data in deepcopy(ldata)

In [ ]:
import numpy as np
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from config import PLOTS_DIR
from utils.attr_dict import AttrDict

betas = np.arange(1, 8)

tstamp = io.get_timestamp('%Y-%m-%d-%H%M')
tint_plots_dir = os.path.join(PLOTS_DIR, 'tau_int_plots', f'{tstamp}')
io.check_else_make_dir(tint_plots_dir)

#cmap = truncate_colormap('binary', minval=0.2, maxval=1.0, n=100)
sns.set_context('paper')
plt.rc('text', usetex=True)

#fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(12, 8), constrained_layout=True)
#fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(9, 5), constrained_layout=True)
fig, axes = plt.subplots(nrows=3, ncols=3, figsize=(6, 4),
                         sharey='row',
                         #sharey='row', sharex='col',
                         constrained_layout=True)
#fig, axes = plt.subplots(nrows=3, ncols=3, constrained_layout=True)
axes = axes.flatten()

cmap = truncate_colormap('binary', minval=0.2, maxval=1., n=num_colors)

axdict = dict(zip(betas, axes))
best_hmc = {
    beta: {} for beta in betas
}
idx = 0
for key, val in outputs.items():
    if val is None:
        continue
    if isinstance(val, list) and len(val) == 1:
        val = val[0]
        
    if val['beta'] not in betas:
        continue
        
        
    lf = val['lf']
    eps = val['eps']
    traj_len = lf * eps
    beta = val['beta']
    
    tint = val.get('tau_int', val.get('tint', None))
    n = val.get('draws', val.get('narr'))
    #n = val['draws']
    #tint = val['tau_int']  # (num_points, batch_size)
    if beta == 5.:
        if np.isnan(tint).any():
            continue
            
    #if 4. < beta < 6.:
    #    if np.max(n) < 1e5:
    #        continue
    
    tint_avg = np.mean(tint, axis=-1)
    tint_err = np.std(tint, axis=-1)
    #x = val['draws']
    x = n
    y = lf * tint_avg
    yerr = lf * tint_err
    #y = * np.mean(tint, axis=-1)
    #yerr = val['lf'] * np.std(tint, axis=-1)# / (np.log(10) * y)
    yend = tint[-1]
    
    if beta == 5. and tint_avg[-1] > 1e4:
        continue
        
    if beta in [5., 6., 7.]:
        keep = np.where(lf * tint_avg > 1e3)
        x = x[keep]
        y = y[keep]
        yerr = yerr[keep]
        
    color = cmap(traj_len / traj_len_max)
    
    
    #if beta == 5.:
    #    console.log(f'beta: {beta}, lf: {lf}, eps: {eps}, {np.max(y)}', style='green')
    #if beta == 6.:
    #    console.log(f'beta: {beta}, lf: {lf}, eps: {eps}, {np.max(y)}', style='cyan')
    #if beta == 7.:
    #    console.log(f'beta: {beta}, lf: {lf}, eps: {eps}, {np.max(y)}', style='#ffff00')
    
    if traj_len > 4.5:
        continue
        
    #if max(x) < 1e5 - 100:
    #    continue
    eps = np.mean(eps)
    
    best_hmc[beta][(lf, eps)] = {
        'traj_len': traj_len,
        'beta': beta,
        'eps': eps,
        'lf': lf,
        'avg': np.mean(yend),
        'err': np.std(yend),# / (np.log(10) * np.mean(tint[-1])),
    }
    
    _ = axdict[beta].errorbar(x, y, yerr=yerr/(np.log(10)),
                              #markersize=8., alpha=0.6,
                              #capsize=3.,
                              marker='.', alpha=0.5, ls='', color=color)
    
    
idx = 0
jdx = 0
best_l2hmc = {
    beta: {
    } for beta in betas
}
idx_dict = {beta: 0 for beta in betas}
tint_l2hmc_dict = {
    beta: {'x': [], 'y': [], 'yerr': []} for beta in betas
}
for key, val in outputs_l2hmc.items():
    if val is None:
        continue
        
    beta = val.get('beta', None)
    if beta is None:
        continue
        
    if isinstance(val, list) and len(val) == 0:
        continue
        
    else:
        #for v in val:
        if 'traj_len' in val.keys():
            traj_len  = val['traj_len']
        else:
            lf = val.get('lf', None)
            eps = tf.reduce_mean(val.get('eps', None))
            if lf is not None and eps is not None:
                traj_len = lf * eps
            else:
                raise ValueError('Unable to determine trajectory length')

        beta = val.get('beta', None)

        tint = val.get('tau_int', val.get('tint', None))
        n = val.get('draws', val.get('narr', None))
        if tint is None and n is None:
            vals_ = list(val.values())
            if isinstance(vals_[0], dict):
                data = vals_[0]
                c1 = 'traj_len' in data
                c2 = 'narr' in data
                c3 = 'tint' in data
                if c1 and c2 and c3:
                    tint = data['tint']
                    n = data['narr']
                
        ###############################################
        #if beta in [5., 6., 7.] and np.max(n) < 8e4:
        #    continue
        ###############################################
        
        #    beta = val['beta']
        #    tint = val['tau_int']
        #    lf = val['lf']
        #    n = val['draws']
        #    eps = tf.reduce_mean(val['eps'])
        #    traj_len = lf * eps
        #    if beta not in betas:
        #        continue

        tint_avg = np.mean(tint, axis=-1)
        tint_err = np.std(tint, axis=-1) # / (np.log(10) * tint_avg)
        tint_max = np.max(tint)

        if beta == 6:
            if tint_avg[0] < 10 or np.isnan(tint_avg[0]) or tint_avg[0] > 50:
                continue

        keep = np.isfinite(tint_avg)
        if beta == 5.:
            keep = np.where(tint_avg < 100)
        if beta == 6.:
            keep = np.where(lf * tint_avg < 800)
        #else:
        #    keep = np.where(tint_avg <  250)

        if beta == 4.:
            if lf > 10:
                continue

        x = n[keep]
        # Scale y and yerr by N_{lf}
        y = lf * tint_avg[keep]
        yerr = lf * tint_err[keep]

        tint_l2hmc_dict[beta]['x'].extend(x)
        tint_l2hmc_dict[beta]['y'].extend(y)
        tint_l2hmc_dict[beta]['yerr'].extend(yerr)

        yend_avg = np.mean(tint[-1])
        yend_err = np.std(tint[-1])

        bkey = (lf, np.mean(eps))
        if bkey in best_l2hmc[beta].keys():
            old_avg = best_l2hmc[beta][bkey]['avg']
            old_err = best_l2hmc[beta][bkey]['err']
            yend_avg = np.mean([old_avg, np.mean(tint[-1])])
            yend_err = np.mean([old_err, np.std(tint[-1])])
            #best_l2hmc[beta][bkey]['avg'] = np.mean([old_avg, np.mean(tint[-1])])
            #best_l2hmc[beta][bkey]['avg'] = np.mean([old_err, np.std(tint[-1])])
            #existing = True
            #small_delta = 1e-12 * np.random.randn()
            #bkey = (lf, np.mean(eps) + small_delta)


        #avgs.append(np.mean(tint[-1]))
        #errs.append(np.std(tint[-1]) / (2.303)# * np.mean(tint[-1])))
        #traj_lens.append(traj_len)

        label = None
        if idx_dict[beta] == 0:
            if beta == 2 or beta == 6.:
                label = f'trained'#, ' + f'$\lambda \simeq {{{traj_len:.3g}}}$'
            else:
                label = f'$\lambda\simeq{{{traj_len:.3g}}}$'
            idx_dict[beta] += 1
        #if idx == 0 and jdx == 0 and beta == 5.:
        #    label = f'L2HMC, $\lambda \simeq {{{traj_len:.2g}}}$'
        #    idx += 1
        #    jdx += 1

        best_l2hmc[beta][bkey] = {
            'beta': beta,
            'lf': lf,
            'eps': eps,
            'traj_len': traj_len,
            'avg': yend_avg,
            'err': yend_err,
        }

        _ = axdict[beta].errorbar(x, y, yerr=yerr / (np.log(10)),
                                  #capsize=3.,
                                  markersize=1.5, alpha=0.6,
                                  marker='s', ls='-', label=label,
                                  #$fillstyle='none',
                                  color='deeppink')
                                  #fillstyle='none', color='deeppink')
            
            
        
#_ = axdict[5.].set_ylabel(r'$N_{\mathrm{LF}} \cdot \tau_{\mathrm{int}}$')
#_ = axdict[5.].legend(loc='upper left', )
#_ = axdict[6.].legend(loc='upper left')
#_ = axdict[7.].legend(loc='upper left')

_ = axdict[2.].set_title(r"$\beta=2$")
_ = axdict[3.].set_title(r"$\beta=3$")
_ = axdict[4.].set_title(r"$\beta=4$")
_ = axdict[5.].set_title(r"$\beta=5$")
_ = axdict[6.].set_title(r"$\beta=6$")
_ = axdict[7.].set_title(r"$\beta=7$")

_ = axdict[2.].set_ylabel(r'$N_{\mathrm{LF}} \cdot \tau_{\mathrm{int}}$')
_ = axdict[5.].set_ylabel(r'$N_{\mathrm{LF}} \cdot \tau_{\mathrm{int}}$')
_ = axdict[6.].set_xlabel('MC Step')
loc_labels = {
    2.: 'upper left',
    3.: 'upper left',
    4.: 'lower right',
    5.: 'upper left',
    6.: 'upper left',
    7.: 'upper left',
}
_ = axdict[2.].legend(loc='upper left', fontsize='small')
for beta, ax in axdict.items():
    _ = ax.set_yscale('log')
    _ = ax.set_xscale('log')
    _ = ax.grid(True, alpha=0.4)
    _ = ax.set_xlim((900, ax.get_xlim()[-1]))
    if beta == 7.:
        _ = ax.set_ylim((100, 120000))
         
    #_ = ax.legend(loc=loc_labels[beta], fontsize='small')
    #if beta > 5.:
    #    _ = ax.set_xlim((ax.get_xlim()[0], 1.01e5))
    
normalize = mcolors.Normalize(vmin=1., vmax=4.)
scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
scalarmappable.set_array(np.unique(tlens_arr))
colorbar = plt.colorbar(scalarmappable, ax=list(axdict.values()), label=r"$\lambda$ (HMC)")
#colorbar.label = r"$\lambda$"
    
out_file = os.path.join(tint_plots_dir, f'tau_int_vs_draws_scaled.pdf')
console.log(f'Saving figure to: {out_file}.')
plt.savefig(out_file, dpi=400, bbox_inches='tight')

In [ ]:
l2hmc_log_dirs = {}
for ldir in l2hmc_dirs:
    inf_idx = str(ldir).find('inference')
    if inf_idx != -1:
        log_dir = str(ldir)[:inf_idx]
        
    ldd_idx = str(ldir).find('LOADED')
    if ldd_idx != -1:
        log_dir = str(ldir)[:ldd_idx]
        
    l2hmc_log_dirs[str(ldir)] = log_dir
    #l2hmc_log_dirs.append(log_dir)
    
    #_, (bi1, bf1) = get_annealing_schedule(str(ldir))
    #try:
    #    bi2, bf2 = search_for_annealing_schedule(str(ldir))
    #except FileNotFoundError:
    #    bi2 = None
    #    bf2 = None
    #    console.log(ldir)
    #    if bi2 != bi1:
    #        console.log(f'bi1: {bi1}, bi2: {bi2}')
    #    if bf1 != bf2:
    #        
    #        console.log(f'bf1: {bf1}, bf2: {bf2}')
    

In [ ]:
import shutil

scheds = {}
for run_dir, log_dir in l2hmc_log_dirs.items():
    sched_file = os.path.join(log_dir, 'annealing_schedule.txt')
    if os.path.isfile(sched_file):
        console.log(f'copying: {sched_file} to {run_dir}.')
        _ = shutil.copy2(sched_file, run_dir)
        sched = np.loadtxt(sched_file, delimiter=',')
        #console.log(f'run_dir: {run_dir}')
        #console.log(f'log_dir: {log_dir}')
        console.log(f'bi: {sched[0]}, bf: {sched[1]}')
        console.rule()

In [ ]:
l2hmc_log_dirs

In [ ]:
bi, bf = search_for_annealing_schedule(str(l2hmc_dirs[0]))

In [ ]:
bi

In [ ]:
bf

In [ ]:
l2hmc_dirs[0]

In [ ]:
bi, bf = get_annealing_schedule(str(l2hmc_dirs[0]))
bi, bf = get_annealing_schedule(str(l2hmc_dirs[0]))

In [ ]:
bi, bf

In [ ]:
for beta in betas:
    console.rule(f'beta: {beta}, num_bad_dirs: {len(list(bad_data_l2hmc[beta].keys()))}')
    for key, val in bad_data_l2hmc[beta].items():
        console.log(f"run_dir: {val['run_params']['run_dir']}")

In [ ]:
np.log(ymin)

In [ ]:
y.shape

In [ ]:
np.min(y, axis=0).shape

In [ ]:
plt.style.use('default')
sns.set_context('paper')
sns.set_palette('bright')
plt.rc('text', usetex=True)

In [ ]:
def get_annealing_schedule(run_dir):
    tail = str(run_dir).find('GaugeModel_logs')
    run_dir = run_dir[tail:]
    
    bidx = str(run_dir).find('bi')
    bfdx = str(run_dir).find('bf')
    
    bistr = run_dir[bidx:bidx+3]
    bfstr = run_dir[bfdx:bfdx+3]
    
    bi = float(bistr.lstrip('bi'))
    bf = float(bfstr.lstrip('bf'))
    
    astr = r"$\beta:$" + f' {bi}' + r"$\rightarrow$" + f'{bf}'
    
    return astr, (bi, bf)

In [ ]:
from cycler import cycler
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from config import PLOTS_DIR, MARKERS
from utils.plotting_utils import truncate_colormap, set_size

datestr = io.get_timestamp('%Y-%m-%d')
timestr = io.get_timestamp('%H%M')
outdir = os.path.join(PLOTS_DIR, f'autocorrs_{datestr}')
io.check_else_make_dir(outdir)

sns.set_context('paper')
sns.set_palette('bright')
plt.rc('text', usetex=True)

betas = np.arange(1, 8)
titles = {
    1.: r"$\beta = 1$",
    2.: r"$\beta = 2$",
    3.: r"$\beta = 3$",
    4.: r"$\beta = 4$",
    5.: r"$\beta = 5$",
    6.: r"$\beta = 6$",
    7.: r"$\beta = 7$",
}
set1_colors = [(228,26,28), (55,126,184), (77,175,74), (152,78,163),  (255,127,0), (255,255,51), (166,86,40), (247,129,191), (153,153,153)]
#set1_colors = np.array(set1_colors, dtype=float)
#set1_colors /= 255.0
#set1_colors = mcolor.to_rgba_array(set1[::2])
alt_colors = ["#7F3C8D", "#11A579", "#3969AC", "#F2B701", "#E73F74", "#80BA5A", "#E68310", "#A5AA99"]
default = ['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
colors = 10 * ['deeppink', 'C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8']

pink_yellow = ["#fef6b5", "#ffdd9a", "#ffc285", "#ffa679", "#fa8a76", "#f16d7a", "#e15383"]
sunset = ["#f3e79b", "#fac484", "#f8a07e", "#eb7f86", "#ce6693", "#a059a0", "#5c53a5"]
plt.style.use('default')
sns.set_style('whitegrid')
sns.set_context('paper')
sns.set_palette('bright')
plt.rc('text', usetex=True)

markers = 10 * ['s', 'd', '^', 'v', '<', '>', 'o', '*']
#cycler('color', ['deeppink', 'dodgerblue', 'lawngreen', 'violet', 'orangered', 'deepskyblue', 'slategrey', 'aquamarine']) #'cyan', 'm', 'y', 'k'])

tint_tlen_hmc = {beta: {} for beta in betas}
tint_tlen_l2hmc = {beta: {} for beta in betas}
bad_l2hmc_run_dirs = {beta: {} for beta in betas}

for beta in betas:
    fig, ax = plt.subplots(figsize=set_size(), constrained_layout=True)
        
    htlens = []
    htdata = dict(sorted(hdata[beta].items()))
    cmap = truncate_colormap('cet_gray', minval=0.1, maxval=0.9, n=len(list(htdata.keys())))
    for idx, (traj_len, ht) in enumerate(htdata.items()):
        lf = ht['lf']
        x = ht['narr']
        y = lf * ht['tint']
        yavg = np.mean(y, axis=-1)
        yerr = np.std(y, axis=-1) / np.log(10)
        ymin = np.min(y, axis=-1)
        ymax = np.max(y, axis=-1)
        
        if beta == 4. and np.max(x) > 5e4:
            continue
        
        keep = np.isfinite(yavg)
        x = x[keep]
        y = y[keep]
        yerr = yerr[keep]
        htlens.append(traj_len)
        
        if beta == 7.:
            keep = np.squeeze(np.where(yavg > 1000))
            x = x[keep]
            y = y[keep]
            yerr = yerr[keep]
            
        tint_tlen_hmc[beta].update({
            traj_len: {
                'x': x,
                'y': y,
                'yerr': yerr,
                'params': ht.get('run_params', None),
            }
        })
        
        color = cmap(traj_len / 4.)
        label = None
        if idx == 0:
            label = r'$\lambda = $' + f'{traj_len:.3g}'
            label += ' (HMC)'
            
        _ = ax.errorbar(x, yavg, yerr=yerr, ls='', marker='.', markersize=2.5, fillstyle='none', color=color)
        _ = ax.fill_between(x, y1=ymin, y2=ymax, color=color, alpha=0.2, zorder=-idx)
        #mec = cmap((traj_len - 0.25 * traj_len)/ 4.)
        #label = None
        #for jdx in range(10):
        #    if jdx == 0 and idx == 0:
        #        label = r'$\lambda = $' + f'{traj_len:.3g}'
        #        label += ' (HMC)'
        #        
        #    _ = ax.errorbar(x, y[:, jdx], yerr=yerr[:, jdx],
        #                    ls='', marker='.',
        #                    markersize=2.5,
        #                    fillstyle='none',  #mec=mec,
        #                    color=color)
        #                    #label=label)
            
    normalize = mcolors.Normalize(vmin=1., vmax=4.)
    scalarmappable = cm.ScalarMappable(norm=normalize, cmap=cmap)
    scalarmappable.set_array(np.unique(htlens))
    #fig.colorbar(im, orientation="horizontal", pad=0.2)

    colorbar = fig.colorbar(scalarmappable, ax=ax, label=r"$\lambda$ (HMC)")
    
    ltdata = dict(sorted(deepcopy(data_l2hmc[beta]).items()))
    #cmap = truncate_colormap('viridis', minval=0., maxval=1., n=len(list(ltdata.keys())))
    #maxval = np.max(list(ltdata.keys()))
    for idx, (traj_len, lt) in enumerate(ltdata.items()):
        if traj_len < 1.:
            continue
            
        if beta == 6.:
            if traj_len < 1.78:
                continue
            
        run_dir = lt['run_params']['run_dir']
        astr, (bi, bf) = get_annealing_schedule(run_dir)
        
        if bf != beta:
            console.log(f'bf != beta ???? (bf: {bf}, beta: {beta})', style=RED)
            continue
        
        if beta == 5.:
            if np.isclose(traj_len, 1.55, 0.01) or np.isclose(traj_len, 1.92, 0.01):
                bad_l2hmc_run_dirs[beta].update({
                    traj_len: {
                        'params': lt.get('run_params', None),
                    },
                })
                continue
                
        if beta == 6.:
            c1 = np.isclose(traj_len, 1.7846, 0.001)
            c2 = np.isclose(traj_len, 1.8226, 0.001)
            c3 = np.isclose(traj_len, 1.8414, 0.001)
            if c1 or c2 or c3:
                bad_l2hmc_run_dirs[beta].update({
                    traj_len: {
                        'params': lt.get('run_params', None),
                    }
                })
                continue
        
        lf = lt['lf']
        x = lt['narr']
        
        #y = lf * lt['tint']
        #yerr = lf * lt['tint'] / np.log(10)
        y = lf * lt['tint']
        yavg = np.mean(y, axis=-1)
        yerr = np.std(y, axis=-1)
        ymin = np.min(y, axis=-1)
        ymax = np.max(y, axis=-1)
        
        label = None
        if idx == 0:
            label = ', '.join([
                r'$\lambda = $' + f'{traj_len:4.3g}, ',
                f'{astr}'
            ])
        
        _ = ax.errorbar(x, yavg, yerr=yerr, marker=markers[idx], markersize=2.5,
                        color=colors[idx], label=label)
        _ = ax.fill_between(x, y1=ymin, y2=ymax, color=colors[idx], alpha=0.2, zorder=-idx)
        
        #try:
        #    keep = np.squeeze(np.where(y < 10000))
        #    #x = x[keep[0, :]]
        #    y = y[keep]
        #    yerr = yerr[keep]
        #   
        #except ValueError:
        #    continue
        #    
        #keep = np.isfinite(y)
        ##x = x[keep[0]]
        #y = y[keep]
        #yerr = yerr[keep]
        
        #keep = np.where(x > 1000)
        ##x = x[keep]
        #y = y[keep]
        #yerr = yerr[keep]
        
        tint_tlen_l2hmc[beta].update({
            traj_len: {
                'x': x,
                'y': y,
                'yerr': yerr,
                'params': ht.get('run_params', None),
            },
        })
        
        #mcolor.Normalize(vmin=0, vmax=1.jkjkjkj)
        
        #for jdx in range(10):
        #    label = None
        #    if jdx == 0 and idx == 0:
        #        label = ', '.join([
        #            r'$\lambda = $' + f'{traj_len:4.3g}, ',
        #            f'{astr}'
        #        ])
        #        #if beta == 6.:
        #        #    label = r'$\lambda = $' + f'{traj_len:6.5g}, '
        #    _ = ax.errorbar(x, y[:, jdx], yerr=yerr[:, jdx],
        #                    ls='', marker=markers[idx],
        #                    markersize=2.5,
        #                    color=colors[idx],
        #                    #color=cmap(traj_len),
        #                    #color='',
        #                    #fillstyle='none',
        #                    label=label)
        
    _ = ax.set_title(titles[beta])
    _ = ax.legend(loc='best', fontsize='x-small')
    #_ = ax.set_xlim((900, ax.get_xlim()[-1]))
    _ = ax.set_yscale('log')
    _ = ax.set_xscale('log')
    _ = ax.set_ylabel(r"$N_{\mathrm{LF}}\cdot \tau_{\mathrm{int}}$")
    _ = ax.set_xlabel('MC Step')
    _ = ax.grid(True, alpha=0.4)
    
    
    outfile = os.path.join(outdir, f'autocorr_beta{int(beta)}.pdf')
    _ = plt.savefig(outfile, dpi=400, bbox_inches='tight')

In [ ]:
%debug

In [ ]:
%debug

In [ ]:
for beta in betas:
    fig, ax = plt.subplots(figsize=set_size(), constrained_layout=True)
        
    cmap = truncate_colormap('cet_gray', minval=0.1, maxval=0.9, n=len(list(htdata.keys())))
    
    hdata_ = dict(sorted(tint_tlen_hmc[beta].items()))
    
    for idx, (traj_len, ht) in enumerate(hdata_.items()):
        x = ht['x']
        y = ht['y']
        yerr = ht['yerr']
        num_steps, batch_size = y.shape
        xtlen = batch_size * [traj_len]
        ytlen = y[-1]
        yerrtlen = yerr[-1]
        _ = ax.errorbar(xtlen, ytlen, yerr=yerrtlen,
                        ls='', marker='.',
                        markersize=2.5,
                        fillstyle='none',  #mec=mec,
                        color=color)
                        #label=label)

In [ ]:
%debug

In [ ]:
%debug

In [ ]:
import utils.autocorr as acr

betas = np.arange(1, 8)
ltint = {
    b: {} for b in betas
}
for idx, ldir in enumerate(l2hmc_dirs):
    console.log(f'({idx}/{len(l2hmc_dirs)}) ldir: {ldir}')
    data = acr._deal_with_new_data
    data = acr.deal_with_new_data(ldir, save=False)
    beta = data['beta']
    ltint[beta].update(data)

In [ ]:
for key, val in ltint.items():
    console.rule(f'beta: {beta}')
    console.log(val)

In [ ]:
ltint[2.].keys()

In [ ]:
for beta, tint in ltint.items():
    fig, ax = plt.subplots()
    _ = ax.errorbar(tint['narr'], np.mean(tint['tint'], axis=-1), yerr=np.std(tint['tint'], axis=-1))

In [ ]:
data